In [1]:
import initialize_notebook
import pandas as pd

from ams.DateRange import DateRange
from ams.services.equities.EquityFundaDimension import EquityFundaDimension
%matplotlib inline
from pathlib import Path

from pyspark.sql import functions as F
from pyspark.sql import types as T

from ams.config import constants
from ams.services import spark_service
from ams.services import twitter_service
from ams.services import ticker_service

from statistics import mean
from sklearn.metrics import accuracy_score
import torch
import numpy as np
from ams.services import file_services
from typing import List

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


Setting up logging...
Will use logging path: C:\Users\Chris\workspaces\data\logs\alpha_media_signal


In [2]:
twitter_folder = 'twitter'

learning_prep_dir = Path(constants.DATA_PATH, twitter_folder, "learning_prep_drop", "learning_prep_drop_2020-10-25_18-00-45-267.63")
# learning_prep_dir = Path(constants.OVERFLOW_DATA_PATH, twitter_folder, "learning_prep_drop", "learning_prep_drop_2020-08-31_03-22-33-670.81")

source_path = file_services.get_single_file_from_path(learning_prep_dir, ".csv")

In [3]:
%%time

df_twitter_raw = pd.read_csv(str(source_path))
df_twitter_raw = df_twitter_raw.sample(frac=.4)

max_date = df_twitter_raw["date"].max()
print(f"Max date: {max_date}")


<string>:2: DtypeWarning: Columns (9,13,16,23,26,31) have mixed types.Specify dtype option on import or set low_memory=False.


Max date: 74.924995
Wall time: 38 s


In [4]:
%%time

df_booled = twitter_service.convert_to_bool(df=df_twitter_raw)

Wall time: 1min 10s


In [5]:
%%time
df_twitter = df_booled[df_booled["f22_num_other_tickers_in_tweet"] == 0]

def coerce_convert_to_numeric(df: pd.DataFrame, col: str):
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(0)
    return df

df_twitter = coerce_convert_to_numeric(df=df_twitter, col="user_followers_count")
df_twitter = coerce_convert_to_numeric(df=df_twitter, col="f22_sentiment_compound")
df_twitter = coerce_convert_to_numeric(df=df_twitter, col="f22_compound_score")

df_stock_data = twitter_service.get_stock_data_for_twitter_companies(df_tweets=df_twitter)

print(df_stock_data.shape[0])

C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


22478
Wall time: 26.7 s


In [6]:
# df_rec_quart_drop = df_rec_quart.drop(columns=["lastupdated", "dimension", "calendardate", "datekey", "reportperiod"])
df_rec_quart_drop = twitter_service.get_rec_quarter_for_twitter()
columns_fundy = list(df_rec_quart_drop.columns)

In [7]:
%%time
# df_stock_and_quarter = pd.merge(df_stock_data, df_rec_quart_drop, how='inner', left_on=["ticker"], right_on=["ticker"])

from ams.services.equities import equity_fundy_service as efs
df_equity_fundies = efs.get_equity_fundies()

df_eq_fun_quarters = df_equity_fundies[df_equity_fundies["dimension"] == EquityFundaDimension.AsReportedQuarterly.value]
df_equity_fundies = None

df_result = pd.merge(df_stock_data, df_eq_fun_quarters, on="ticker", suffixes=[None, "_eq_fun"]).sort_values(by=["calendardate"])
df_eq_fun_quarters = None
df_stock_data = None

df_drop_future = df_result[df_result["date"] > df_result["calendardate"]]

df_stock_and_quarter = df_drop_future.drop_duplicates(subset=["ticker", "date"], keep="last").drop(columns=["lastupdated_eq_fun", "reportperiod", "calendardate", "dimension", "datekey"])
df_drop_future = None

df_stock_and_quarter.shape[0]

Wall time: 8.37 s


20479

In [8]:
df_nas_tickers_info = ticker_service.get_nasdaq_tickers()

C:\Users\Chris\workspaces\alpha_media_signal\ams\services\ticker_service.py:254: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = df[c].fillna("<unknown>")
C:\Users\Chris\workspaces\alpha_media_signal\ams\services\ticker_service.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = df[c].astype(CategoricalDtype(uniques))


In [9]:
%time

col_ticker = "ticker_drop"

df_stock_quart_info = pd.merge(df_stock_and_quarter, df_nas_tickers_info, how='inner', left_on=["ticker"], right_on=[col_ticker])
df_nas_tickers_info = None
df_sqi = df_stock_quart_info.drop(columns=[col_ticker])
df_stock_quart_info = None


print(df_sqi.shape)

Wall time: 0 ns
(40958, 584)


In [10]:
%%time

df_stock_renamed = df_sqi.rename(columns={"ticker": "f22_ticker"})

if 'None' in df_stock_renamed.columns:
    df_stock_renamed = df_stock_renamed.drop(columns=['None'])

print(f'Twitter: {df_twitter.shape[0]}')
print(f'Quandl: {df_stock_renamed.shape[0]}')

print(f'{df_twitter.columns}')

df_merged = pd.merge(df_twitter, df_stock_renamed, how='inner', left_on=["f22_ticker","date"], right_on=["f22_ticker","date"])
df_twitter = None

print(f'Num merged: {df_merged.shape[0]}')

Twitter: 1515357
Quandl: 40958
Index(['created_at', 'created_at_timestamp', 'user_screen_name', 'favorite_count', 'in_reply_to_screen_name', 'user_verified', 'metadata_result_type', 'user_listed_count', 'user_time_zone', 'user_has_extended_profile', 'in_reply_to_status_id', 'user_statuses_count', 'user_protected', 'user_is_translation_enabled', 'user_location', 'lang', 'user_geo_enabled', 'place_country', 'place_name', 'possibly_sensitive', 'user_friends_count', 'retweet_count', 'user_follow_request_sent', 'user_followers_count', 'f22_ticker', 'f22_has_cashtag', 'f22_ticker_in_text', 'f22_num_other_tickers_in_tweet', 'f22_sentiment_pos', 'f22_sentiment_neu', 'f22_sentiment_neg', 'f22_sentiment_compound', 'date', 'f22_compound_score'], dtype='object')
Num merged: 876212
Wall time: 11 s


In [11]:
%%time

df_thin_rabbit = twitter_service.add_buy_sell(df=df_merged)
df_thin_rabbit["original_close_price"] = df_thin_rabbit["close"]

Wall time: 297 ms


In [12]:
%%time
cols_fundy_numeric = list(set(columns_fundy) - {"ticker", 'calendardate', 'datekey', 'reportperiod'})

df_no_z = twitter_service.fill_null_numeric(df=df_thin_rabbit, cols_fundy_numeric=cols_fundy_numeric)

Wall time: 2.62 s


In [13]:
%%time
df_thin_rabbit = None
df_merged = None

df_green_hornet = twitter_service.fill_empty_str_cols(df=df_no_z)

Wall time: 7.08 s


In [14]:
%%time

df_after_hours = twitter_service.add_is_tweet_after_hours(df=df_green_hornet)

Wall time: 1min 21s


In [15]:
%%time
from ams.utils import ticker_utils

df_sma = ticker_utils.add_sma_history(df=df_after_hours, target_column="close", windows=[15, 20, 50, 100, 200])

Wall time: 35.9 s


In [16]:
%%time
df_since_sma = ticker_utils.add_days_since_under_sma_many_tickers(df=df_sma, col_sma="close_SMA_200", close_col="close")
df_since_sma = ticker_utils.add_days_since_under_sma_many_tickers(df=df_since_sma, col_sma="close_SMA_15", close_col="close")
df_since_sma = ticker_utils.add_days_since_under_sma_many_tickers(df=df_since_sma, col_sma="close_SMA_20", close_col="close")
df_since_sma = ticker_utils.add_days_since_under_sma_many_tickers(df=df_since_sma, col_sma="close_SMA_50", close_col="close")
df_since_sma = ticker_utils.add_days_since_under_sma_many_tickers(df=df_since_sma, col_sma="close_SMA_100", close_col="close")

Wall time: 2min 58s


In [17]:
%%time

df_pay = twitter_service.add_purchase_date(df=df_since_sma)

Wall time: 1.31 s


In [18]:
%%time

df_days_until = ticker_service.add_days_until_sale(df=df_pay)

Wall time: 53 s


In [19]:
class SplitData():
    X_train = None
    y_train = None
    X_test = None
    y_test = None
    df_test_raw = None
    df_test_std = None
    train_cols = None
    has_enough_data = None
    
    def __init__(self, 
                 X_train: np.array,
                 y_train: np.array,
                 X_test: np.array,
                 y_test: np.array,
                 df_test_raw: pd.DataFrame,
                 df_test_std: pd.DataFrame,
                 df_val_raw: pd.DataFrame,
                 df_val_std: pd.DataFrame,
                 train_cols: List[str],
                 has_enough_data: bool
                ):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.df_test_raw = df_test_raw
        self.df_test_std = df_test_std
        self.df_val_raw = df_val_raw
        self.df_val_std = df_val_std
        self.train_cols = train_cols
        self.has_enough_data = has_enough_data

In [20]:
def has_f22_column(df: pd.DataFrame):
    cols = [c for c in df.columns if c.startswith("f22")]
    print(cols)

def split_off_data(df: pd.DataFrame, narrow_cols: List[str], tickers: List[str] = None) -> SplitData :
    use_recent_for_holdout = True
    df_samp, df_val_raw = twitter_service.ho_split_by_days(df, 
                                                            small_data_days_to_pull=None, 
                                                            small_data_frac=.025, 
                                                            use_only_recent_for_holdout=use_recent_for_holdout)
    # df_samp, df_val_raw = twitter_service.ho_split_by_ticker(df)
    # df_samp, df_val_raw = twitter_service.ho_split_by_date(df)
    # print(f"Will use: {tickers[:10]}.")
    # df_samp, df_val_raw = twitter_service.ho_split_by_tickers(df=df, tickers=tickers)
    # has_enough_data = df_samp is not None and df_val_raw is not None
        
    X_train = None
    y_train = None
    X_test = None
    y_test = None
    df_test_raw = None
    df_test_std = None
    df_val_std = None
    train_cols = None
    
    has_enough_data = df_samp is not None and df_val_raw is not None and df_val_raw.shape[0] > 500
    if has_enough_data:

        df_shuff = df_samp.sample(frac=1.0)

        df_train_raw, df_test_raw = twitter_service.ho_split_by_days(df_shuff, small_data_days_to_pull=None, small_data_frac=.2, use_only_recent_for_holdout=use_recent_for_holdout) #        

        has_enough_data = df_train_raw is not None and df_test_raw is not None and df_test_raw.shape[0] > 500
        if has_enough_data:
            print(f"Original: {df.shape[0]}; train_set: {df_train_raw.shape[0]}; test_set: {df_test_raw.shape[0]}")

            df_train_std, df_test_std, df_val_std = ticker_service.std_dataframe(df_train=df_train_raw, df_test=df_test_raw, df_val=df_val_raw)
            

            X_train, y_train, X_test, y_test, train_cols = twitter_service.split_train_test(train_set=df_train_std, test_set=df_test_std, narrow_cols=narrow_cols)

        return SplitData(X_train=X_train,
                       y_train=y_train,
                       X_test=X_test,
                       y_test=y_test,
                       df_test_raw=df_test_raw,
                       df_test_std=df_test_std,
                       df_val_raw=df_val_raw,
                       df_val_std=df_val_std,
                       train_cols=train_cols,
                       has_enough_data=has_enough_data)

In [21]:
def group_and_mean_preds(df: pd.DataFrame, model: object, train_cols: List[str], is_model_torch=False):
    df_g_holdout = df.groupby(['f22_ticker', 'purchase_date'])

    group_count = 0
    acc_acc = []
    group_preds = {}
    for group_name, df_group in df_g_holdout:
        X_holdout = np.array(df_group[train_cols])
        y_holdout = np.array(df_group['buy_sell'])
        
        if is_model_torch:
            X_holdout_con = X_holdout.astype(desired_dtype)
            X_torch = torch.FloatTensor(X_holdout_con)
            
            y_holdout = np.where(y_holdout==-1, 0, y_holdout)
            pre_y_ho = sum(y_holdout)/len(y_holdout)
            pre_y_ho = 0 if pre_y_ho < .5 else 1
            
            model.eval()

            with torch.no_grad():
                model.cpu()
                raw_out = model(X_torch)
            
            prediction = raw_out.data.numpy()
            y_pred_tag = torch.round(torch.sigmoid(raw_out.data))
            pred_mean = sum(y_pred_tag.numpy()) / len(y_holdout)

            pred_mean = -1 if pred_mean < .5 else 1
        else:
            prediction = model.predict(X_holdout)

            pred_mean = sum(prediction)/len(prediction)
            pred_mean = -1 if pred_mean < 0 else 1

            pre_y_ho = sum(y_holdout)/len(y_holdout)
            pre_y_ho = -1 if pre_y_ho < 0 else 1

        # NOTE: This means that we are predicting only when buys are successfull. And we only buy.
        pred_buy_success = 0
        if pred_mean == 1:
            if pre_y_ho == pred_mean:
                pred_buy_success = 1
            
            # Of the buys what's is the success rate?
            acc_acc.append(pred_buy_success)

        ticker = group_name[0]
        date_str = group_name[1]
        if ticker in group_preds.keys():
            info = group_preds[ticker]
        else:
            info = {}
            group_preds[ticker] = info
        info[date_str] = pred_mean

    if len(acc_acc) > 0:
        print(f"Mean: {mean(acc_acc)}")
    else:
        print("No mean on grouped mean - no rows?")

    g_tickers = []
    for group_name, df_group in df_g_holdout:
        g_tickers.append(group_name[0])
    
    return g_tickers, group_preds

In [22]:
initial_cash = 10000
num_trades_at_once = 1
max_trades = 5000

def calc_profit(target_roi: float, df_helper: pd.DataFrame, group_preds: object, zero_in: bool = False):
    roi_list = []
    trade_cost = 0
    pred_correct = 0
    pot_roi_list = []
    calc_roi_list = []
    num_trades = 0
    cash = initial_cash
    calc_dict = {}
    total = 0
    total_invested = 0
    fc_num_trades = 0
    for ticker, info in group_preds.items():
        num_shares = 0
        action = "<not specified yet>"
        
        for date_str, pred_mean in info.items():
            split_share_multiplier = 1
            should_buy = (pred_mean == 1)
            
            close, future_close, future_high, future_open, split_share_multiplier = ticker_service.get_stock_info(df=df_helper, \
                                                                                                                  ticker=ticker, \
                                                                                                                  date_str=date_str)
            num_shares = split_share_multiplier * num_shares

            pot_roi = (future_high - close) / close
            pot_roi_list.append(pot_roi)
            
            shares_price = num_trades_at_once * close
            
            if should_buy:
                ticker_service.calculate_roi(target_roi=target_roi, close_price=close, future_high=future_high, future_close=future_close, calc_dict=calc_dict, zero_in=zero_in)
                
                roi = (future_close - close) / close
                roi_list.append(roi)
                
                if future_close > close:
                    pred_correct += 1
                fc_num_trades += 1
                
                if cash > shares_price:
                    cash -= shares_price + trade_cost
                    num_shares += num_trades_at_once
                    num_trades += 1

                    total_invested += shares_price + trade_cost
                    cashed_out = ((split_share_multiplier * num_shares) * future_close) - trade_cost
                    total = cashed_out + cash
                    cash = total
                    num_shares = 0
                else:
                    print("Not enough cash for purchase.")

            if num_trades > max_trades:
                break

        if num_trades > max_trades:
                break
    
    acc = -1
    if fc_num_trades > 0:
        acc = pred_correct/fc_num_trades
        
    return total, num_trades, acc, calc_dict, roi_list, pot_roi_list, total_invested

In [23]:
def get_roi_matrix(df: pd.DataFrame, group_preds: object, target_roi_frac: float, zero_in: bool = False):
    total, num_trades, acc, calc_dict, roi_list, pot_roi_list, total_invested = calc_profit(target_roi=target_roi_frac, \
                                                                                            df_helper=df, \
                                                                                            group_preds=group_preds, \
                                                                                            zero_in = zero_in)
   
    sac_roi = None
    if len(roi_list) > 0:
        sac_roi = mean(roi_list)
    
    profit = total - initial_cash
    
    if num_trades > 0:
        profit_per_trade = round(profit / num_trades, 6)
        investment_per_trade = total_invested / num_trades
        roi_per_trade = round(profit_per_trade / investment_per_trade, 4)
        print(f"Avg investment per trade: {investment_per_trade}")
        print(f"Roi per trade: {roi_per_trade}")

    pot = round(sum(pot_roi_list)/len(pot_roi_list), 6)
    if acc > 0:
        acc = round(acc, 5)
    else:
        acc = "<NA>"
    
    print(f"Num trades: {num_trades} | acc: {acc} | s@close roi: {sac_roi} | s@high roi: {pot}")

    avg_list = []
    for roi_calc, roi_calc_list in calc_dict.items():
        avg_roi = mean(roi_calc_list)
        num_list = len(roi_calc_list)
        print(f"Sell high/close roi@{roi_calc}: {round(avg_roi, 6)}; weight: {num_list * avg_roi}")
        avg_list.append(avg_roi)
    
    return avg_list, sac_roi

In [24]:
%%time

min_volume = 100000
min_price = 5.00

df = twitter_service.refine_pool(df=df_days_until, min_volume=min_volume, min_price=min_price)
df = twitter_service.omit_columns(df=df)
df_tweet_counted = twitter_service.add_tweet_count(df=df)

df = None
narrow_cols = list(df_tweet_counted.columns)

Wall time: 3min 2s


In [25]:
from matplotlib.legend_handler import HandlerLine2D
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import numpy as np

def randy_forest(X_train: np.array, y_train: np.array, X_test: np.array, y_test: np.array):
    n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
    train_results = []
    test_results = []

    for estimator in n_estimators:
        rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
        rf.fit(X_train, y_train)
        train_pred = rf.predict(X_train)
        from sklearn.metrics import roc_curve
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
        from sklearn.metrics import auc
        roc_auc = auc(false_positive_rate, true_positive_rate)
        train_results.append(roc_auc)
        y_pred = rf.predict(X_test)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        test_results.append(roc_auc)

    line1, = plt.plot(n_estimators, train_results, 'b', label="Train AUC")
    line2, = plt.plot(n_estimators, test_results, 'r', label="Test AUC")
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel('AUC score')
    plt.xlabel('n_estimators')
    plt.show()

In [26]:
from matplotlib.legend_handler import HandlerLine2D
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import numpy as np

def randy_forest(X_train: np.array, y_train: np.array, X_test: np.array, y_test: np.array):
    n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
    train_results = []
    test_results = []

    for estimator in n_estimators:
        rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
        rf.fit(X_train, y_train)
        train_pred = rf.predict(X_train)
        from sklearn.metrics import roc_curve
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
        from sklearn.metrics import auc
        roc_auc = auc(false_positive_rate, true_positive_rate)
        train_results.append(roc_auc)
        y_pred = rf.predict(X_test)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        test_results.append(roc_auc)

    line1, = plt.plot(n_estimators, train_results, 'b', label="Train AUC")
    line2, = plt.plot(n_estimators, test_results, 'r', label="Test AUC")
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel('AUC score')
    plt.xlabel('n_estimators')
    plt.show()

In [27]:
def rnd_forest_clf(X_train: np.array, y_train: np.array, X_test: np.array, y_test: np.array):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score

    rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
     max_depth=None, max_features='auto', max_leaf_nodes=None,
     min_impurity_split=1e-07, min_samples_leaf=1,
     min_samples_split=2, min_weight_fraction_leaf=0.0,
     n_estimators=16, n_jobs=1, oob_score=False, random_state=None,
     verbose=0, warm_start=False)
    rf.fit(X=X_train, y=y_train)

    y_test_pred = rf.predict(X_test)

    print(accuracy_score(y_test, y_test_pred))

    return rf

In [28]:
%%time
# raise Exception("Add time data Monday, Tuesday, first of month, etc.")
# Add technical details? 20MA, etc?

from ams.utils import date_utils
import random
from datetime import timedelta

def find_ml_pred_perf(df: pd.DataFrame):
    zero_in = True
    if zero_in:
        target_roi_frac = 0.02
    else:
        target_roi_frac = .001

    print(f"Count: {df.shape[0]}")

    use_rnd_forest = True

    all_avgs = []

    tickers = df["f22_ticker"].unique().tolist()
    random.shuffle(tickers)

    sac_roi_list = []

    num_iterations = 40
    count = 0
    while True:
        count += 1
        
        if count > num_iterations:
            break
        
        sd = split_off_data(df=df, narrow_cols=narrow_cols)
        X_train = sd.X_train
        y_train = sd.y_train
        X_test = sd.X_test
        y_test = sd.y_test
        df_test_raw = sd.df_test_raw
        df_test_std = sd.df_test_std
        df_val_raw = sd.df_val_raw
        df_val_std = sd.df_val_std
        train_cols = sd.train_cols
        has_enough_data = sd.has_enough_data

        if not has_enough_data:
            print("No more days to test.")
            break
        df, has_remaining_days = twitter_service.remove_last_days(df=df, num_days=1)
        
        if not has_remaining_days:
            print("No more remaining days to test.")
            break
        if df_test_raw.shape[0] == 0:
            raise Exception("No holdout data.")
       
        if use_rnd_forest:
            
#             randy_forest(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
#             model = rnd_forest_clf(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

            model = twitter_service.dec_tree(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, max_depth=16)            
            acc_model = model
        else:
            clf = twitter_service.train_mlp(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
            acc_model = clf

        df_splitted = twitter_service.join_with_stock_splits(df=df_val_raw)

        g_tickers, group_preds = group_and_mean_preds(df_val_std, acc_model, train_cols)

        roi_list, sac_roi = get_roi_matrix(df=df_splitted, group_preds=group_preds, target_roi_frac=target_roi_frac, zero_in = zero_in)

        if len(roi_list) > 0:
            max_avg = max(roi_list)
            all_avgs.append(max_avg)

        if sac_roi is not None:
            sac_roi_list.append(sac_roi)

        print(f"Cumulative Metrics: Avg Best Sell@High ROI rate: {mean(all_avgs)} | Avg Sell@Close ROI: {mean(sac_roi_list)} ")

# df_4_model = df_tweet_counted #.sample(frac=.4)
# find_ml_pred_perf(df=df_4_model)

Wall time: 0 ns


In [29]:
from torch.utils.data import Dataset
import random
import torch
import torch.nn as nn
import torch.optim as optim
num_iterations = 1
from torch.utils.data import DataLoader
import torch.nn.functional as F

class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


## test data    
class testData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [30]:
import copy
BATCH_SIZE = 16
LEARNING_RATE = .0001
from ams.utils.Stopwatch import Stopwatch

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

def train(model, data_loader, epoch_ndx, criterion: object, optimizer: object):
    model.train()
    train_loss = 0
    epoch_loss = 0
    epoch_acc = 0
    start = Stopwatch(start_now=True)
    for batch_idx, (X_batch, y_batch) in enumerate(data_loader):
        
        optimizer.zero_grad()
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
     
    num_items = len(data_loader)
    avg_loss = epoch_loss/num_items
    avg_acc = epoch_acc/num_items
    print(f'Epoch {epoch_ndx+0:03}: | Loss: {avg_loss:.5f} | Acc: {avg_acc:.3f}')
    
    start.end(msg="Epoch time")
    
    return avg_loss, avg_acc

def test(model, data_loader, criterion: object):
    model.eval()
    test_loss = 0
    correct = 0
    total_loss = 0
    total_acc = 0
    with torch.no_grad():
        count_loops = 0
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            y_pred = model(X_batch)
        
            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            total_loss += loss.item()
            total_acc += acc.item()
            count_loops += 1
        
    num_items = len(data_loader.dataset)
    avg_loss = total_loss / num_items
    avg_acc = total_acc / count_loops
    
    print(f'\nTest loss: {avg_loss:.4f} | Acc: {avg_acc:.2f}% | Count_loops: {count_loops})\n')
    
    return avg_loss, avg_acc

def train_model(model: object, train_loader: object, test_loader: object, criterion: object, optimizer: object):

    model.train()
    EPOCHS = 200

    last_t_loss = 9999999999
    best_test_loss = 9999999999
    best_test_acc = 0
    countdown = 0
    max_wait_countdown = 6
    for e in range(1, EPOCHS + 1):
        loss, acc = train(model, train_loader, e, criterion=criterion, optimizer=optimizer)
        t_loss, t_acc = test(model, test_loader, criterion=criterion)

        # Overfitting guard
#         if t_loss < best_test_loss:
        if t_acc > best_test_acc:
            best_test_loss = t_loss
            best_test_acc = t_acc
            
            best_model = copy.deepcopy(model)
            countdown = 0
        else:
            countdown += 1

        if countdown > max_wait_countdown:
            break

    print(f"Best loss: {loss} | best acc: {acc}% | best test loss: {best_test_loss} | best test acc: {best_test_acc}%")
    
    return best_model

In [31]:
from typing import Dict

class BinaryClassification(nn.Module):
    def __init__(self, num_input_features: int):
        super(BinaryClassification, self).__init__()
        #Our network consists of 3 layers. 1 input, 1 hidden and 1 output layer
        #This applies Linear transformation to input data. 
        self.fc1 = nn.Linear(num_input_features, num_input_features + 10)
        #This applies linear transformation to produce output data
        self.fc2 = nn.Linear(num_input_features + 10, num_input_features)
        self.layer_out = nn.Linear(num_input_features, 1)
        self.relu = nn.ReLU()
        
    #This must be implemented
    def forward(self, inputs):
        #Output of the first layer
        x = self.relu(self.fc1(inputs))
        x = self.relu(self.fc2(x))
        #This produces output
        x = self.layer_out(x)
        
        return x
    
#     def __init__(self, num_input_features: int):
#         super(BinaryClassification, self).__init__()
#         self.layer_1 = nn.Linear(num_input_features, num_input_features - 10) 
#         self.layer_2 = nn.Linear(num_input_features - 10, num_input_features)
#         self.layer_out = nn.Linear(num_input_features, 1) 
        
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(p=0.1)
#         self.batchnorm1 = nn.BatchNorm1d(num_input_features - 10,)
#         self.batchnorm2 = nn.BatchNorm1d(num_input_features)
        
#     def forward(self, inputs):
#         x = self.relu(self.layer_1(inputs))
#         x = self.batchnorm1(x)
#         x = self.relu(self.layer_2(x))
#         x = self.batchnorm2(x)
#         x = self.dropout(x)
#         x = self.layer_out(x)
        
#         return x

def initialize_gpu():
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.set_device(0)
        device = torch.device('cuda:0')
        print("Using GPU")
    else:
        print("Using CPU")
        device = torch.device('cpu')
    
    return device

def get_initialized_model(num_input_features: int, device: object):
    model = BinaryClassification(num_input_features)
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    return model, criterion, optimizer

def calc_nn_roi(
                df_val_raw: pd.DataFrame, 
                df_val_std: pd.DataFrame, 
                model: object, 
                train_cols: List[str],
                target_roi_frac: float,
                zero_in: bool
               ):

    all_avgs = []
    sac_roi_list = []
    if df_val_raw.shape[0] == 0:
        raise Exception("No holdout data.")

    g_tickers, group_preds = group_and_mean_preds(df_val_std, model, train_cols, is_model_torch=True)

    df_splitted = twitter_service.join_with_stock_splits(df=df_val_raw)

    roi_list, sac_roi = get_roi_matrix(df=df_splitted, group_preds=group_preds, target_roi_frac=target_roi_frac, zero_in = zero_in)

    if len(roi_list) > 0:
        max_avg = max(roi_list)
        all_avgs.append(max_avg)

    if sac_roi is not None:
        sac_roi_list.append(sac_roi)

    target_roi_frac += .001

    if len(all_avgs) > 0 and len(sac_roi_list) > 0:
        print(f"Mean s@high: {mean(all_avgs)} | mean sac_roi: {mean(sac_roi_list)}")
    
    return sac_roi_list

In [32]:
# for c in narrow_cols:
#     dtype = str(df_tweet_counted[c].dtype)
#     if dtype not in ['uint8', 'float64', 'int64', 'int32', 'bool']:
#         print(f"{c}: {dtype}")
# df_tweet_counted.shape[0]

# print(narrow_cols)

# print(len(list(df_tweet_counted["user_screen_name"].unique())))

# df_tweet_counted.sort_values(by="date", ascending=False, inplace=True)
# df_tweet_counted["date"].head(10)
# print(narrow_cols)

# TOP_2019 = ["AMZN", "NFLX", "COP", "EC", "ABMD", "VEEV", "SNP", "CTL", "SQ", "SHOP"]
TOP_2020 = ['PYPL', 'TSLA', 'AMZN', 'CMCSA', 'FB', 'INTC', 'AAPL', 'NVDA', 'MSFT', 'NFLX', 'PEP', 'CSCO', 'ADBE', 'GOOGL', 'WDAY', 'DOCU', 'NBIX', 'AAL', 'TXN', 'TLRY', 'NBL', 'AVGO', 'SIVB', 'WDC', 'MEDI', 'CINF', 'DNA', 'GMST', 'NOVL', 'LKQ', 'CHTR', 'LINE', 'TLSNY', 'SIRI', 'ORLY', 'TMUS', 'ATML', 'ZION', 'GRRF', 'GRPN', 'GRMN', 'PALM', 'SII1', 'ZM', 'GRFS', 'ZNGA', 'MDVN', 'LIFE2', 'GR', 'GILD', 'CHRT', 'CHRW', 'GPRO', 'ATVI', 'MOLX', 'ZS', 'DOX', 'REG', 'REGN', 'IONS', 'DLTR', 'GMCR', 'TIBX', 'BBBY', 'MFNXQ', 'STLD', 'SLM', 'GMAB', 'GH', 'LNT', 'ATHM1', 'ACGL', 'SEDG', 'IPGP', 'ENDP', 'OTEX', 'TIBB', 'SLXP', 'MEL', 'MELI', 'SEBL', 'GENZ', 'BCOR', 'STNE', 'SWKS', 'LOGI', 'ULTA', 'QRTEA', 'CHIR1', 'DIGX', 'RFMD', 'STCN', 'GLPI', 'GLPG', 'ABMD', 'KRFT', 'LLTC', 'DKNG', 'TLAB', 'AYE', 'WBA', 'AZN', 'WB', 'SEIC', 'HK1', 'MNST', 'DISH', 'DISCA', 'CHKP', 'QRVO', 'UB', 'SSNC', 'CMRCQ', 'PCYC', 'TW', 'WWY', 'NTAP', 'WYNN', 'WLTW', 'SRCL', 'MLNM', 'XEL', 'GDS', 'SGEN', 'LBRDA', 'ARMH', 'EBAY', 'LBTYA', 'MCWEQ', 'CG', 'XLNX', 'GEC', 'MLCO', 'HBOC', 'TRB', 'MDB', 'WMG', 'EA', 'CME', 'TCGIA', 'PPD', 'DTV1', 'TTWO', 'TRMB', 'APOL', 'MYL', 'CERN', 'SPWR', 'INKT', 'TRIP', 'DWRI', 'HQCL', 'TVGIA', 'CMVT', 'SHLD', 'NTES', 'EDS', 'DXCM', 'HET', 'TCOMA', 'LCTO', 'TCOM', 'ARBA', 'PDD', 'RATL', 'MKTX', 'RBAK', 'ASO1', 'SRPT', 'YHOO', 'SHPG', 'GTW', 'HNZ', 'NKLA', 'YNDX', 'MDLZ', 'PFG', 'NKTR', 'SIAL', 'Z', 'GT', 'NLOK', 'PAYX', 'ZBRA', 'STOR1', 'UAL', 'INTU', 'WFM', 'ZI', 'ASND1', 'ASML', 'NSOL', 'HBAN', 'XOXO1', 'XP', 'PCAR', 'TWTC', 'WIX', 'GEMP1', 'HOLX', 'HAS', 'RBBN', 'NIHDQ', 'XRAY', 'HAND', 'CGNX', 'XTO', 'GX', 'NIPNY', 'DELL1', 'ACTA', 'FMKT', 'SPLK', 'MAT', 'SBUX', 'EXDS', 'CTAS', 'ALXN', 'SPLS', 'JD', 'CSX', 'AMAT', 'ODP', 'ODFL', 'COMS1', 'AMCC', 'JBHT', 'JAZZ', 'PTON', 'VITR', 'AMD', 'EXPD', 'FLEX', 'EXPE', 'CSGP', 'JAVA', 'EXC', 'ALTR1', 'AMGN', 'EXAS', 'ETSY', 'NTRS', 'CTXS', 'ALGN', 'NDSN', 'ALKS', 'CTSH', 'VMED', 'CTRX', 'OKTA', 'BRCD', 'BRCM', 'SOTR', 'ISRG', 'JKHY', 'ALNY', 'MAR', 'MASI', 'PSFT', 'UVN', 'ETEK1', 'COX', 'PTC', 'FEYE', 'FDRY', 'MCLDQ', 'ANGI', 'SBAC', 'ITWO', 'NXTL', 'NXPI', 'MCDT', 'VEON', 'SANM', 'NWL', 'CPRT', 'CAR', 'CPWR', 'MCIC', 'ANSS', 'MCHP', 'CA', 'NVAX', 'EXTR', 'NVCR', 'FITB', 'FISV', 'VIGN', 'BVSN', 'BVXV', 'CRWD', 'FANG', 'BWNG', 'IYCOY', 'VIAV', 'AMTD', 'COST', 'COUP', 'RYAAY', 'FAST', 'VIAC', 'BYND', 'VIAB', 'FFIV', 'ETFC', 'VOD', 'BGNE', 'FWONA', 'FWLT', 'QDEL', 'UPCOY', 'MERQ', 'BIDU', 'LSI1', 'BIIB', 'BILI', 'QCOM', 'TFCF', 'VRTX', 'EQIX', 'VRTS1', 'STX', 'FTNT', 'BIVV', 'RNWK', 'VSTR1', 'DBX', 'SNDK', 'ULTI', 'KLAC', 'BEAS', 'QLGC', 'SDLI', 'ADCT1', 'QKKAD', 'DDOG', 'ADI', 'IQ', 'VTSS', 'ADP', 'ADSK', 'LRCX', 'DCLK', 'KHC', 'BGEN', 'TEAM', 'LYFT', 'NDAQ', 'BMRN', 'LYO', 'CDW', 'ROST', 'NTNX', 'SNY', 'AKAM', 'BNI', 'ESRX', 'BNTX', 'ON', 'AKZOY', 'RPRX', 'CDNS', 'FOXA', 'FOX1', 'BMET', 'BMC', 'ERIC', 'VRSN', 'VRSK', 'BKNG', 'AFM', 'ERIE', 'ROH', 'TER', 'LULU', 'SUN1', 'SCMR', 'FSLR', 'AGNC', 'CELG', 'ROKU', 'SNPS', 'LVGO', 'TECH', 'ONIS', 'BLUE', 'JUNO', 'NWSA', 'PODD', 'HUNT', 'POOL', 'PLL', 'IEP', 'IDXX', 'IBEM', 'IAC', 'HZNP', 'IMNX', 'MTCH2', 'PMCS', 'IMMU', 'MTCH', 'MRNA', 'INAPQ', 'TROW', 'ILMN', 'MRVL', 'MU', 'MPWR', 'INCY', 'TSCO', 'TSRO', 'HSIC', 'MXIM', 'TSIX1', 'PGN1', 'TTD', 'HTHT', 'ISIL', 'IIXL', 'APAGF', 'SVNX', 'AONEQ', 'EXPO', 'ISHP1', 'MCIP', 'CVBF', 'EXXIQ', 'APCC', 'ESPR', 'MRX', 'ARRY1', 'BRKR', 'ITMR', 'API', 'MDCO', 'ESLT', 'MRVC', 'EXFO', 'ARTG', 'MLHR', 'SNCR', 'EXEL', 'STAY', 'MIPS', 'IDTI', 'ITMN', 'MINI', 'TMTA', 'CY', 'BRKS', 'MLNX', 'ISCA', 'PLXS', 'MSLV', 'IDIX', 'SNDA', 'EXLS', 'BOBJ', 'EXNT1', 'CYCL', 'SNDE', 'ITCD', 'SSTI1', 'SSSW', 'MKSI', 'ISBC', 'FCFS', 'EXPEA', 'MUSE', 'IRWD', 'ASCL', 'MEDP', 'SNDV', 'FCEL', 'SODA', 'ARUN', 'BUCY', 'ESPRY', 'MDGL', 'ITRI', 'BOKF', 'STAA', 'ESGR', 'CYBR', 'ARVN', 'PSEC', 'SSYS', 'ISEE', 'ARWR', 'BUFF', 'BRL', 'IKN', 'AQNT', 'IIXC', 'STRA', 'IIVI', 'APPB', 'EVEP', 'ARCE', 'MDSO', 'EVBG', 'IGTE', 'ARCC1', 'CWTRQ', 'PRXL', 'CWST', 'PLUG', 'MDRX', 'ARGX', 'ARIA', 'ETYS', 'BPOP', 'SNRA', 'STRZA', 'EWBC', 'CWCIQ', 'MSTR', 'BPY', 'BPYU', 'ARRS', 'TMWD', 'PSMT', 'CZR2', 'EVOQ', 'EVOP', 'CZR1', 'CZR', 'TMCS', 'APPX', 'DADA', 'SVC', 'DADE', 'MCTR', 'APPN', 'SUSQ', 'IGMS', 'STRM1', 'APPF', 'PS', 'ARCC', 'SVMK', 'STSA1', 'STEI', 'ISSX', 'TNOX', 'ARNA', 'IRTC', 'ISLD', 'MCRL', 'STMP', 'STXN', 'APLS', 'EYE', 'BPMC', 'SWKH', 'ARPT', 'DCPH', 'SUPN', 'MSCC', 'TLND', 'CVET', 'APKT', 'CVCO', 'DCEL', 'MCRS', 'ITCI', 'FATE', 'STTSY', 'TNDM', 'ARLP', 'MTFB', 'SWCM', 'PSIXQ', 'ETEC1', 'CXRX', 'IFIN', 'SMTC', 'TLTOA', 'CYTC', 'SNSAY', 'STLW', 'MTSI', 'CVLT', 'SVVS', 'INGN', 'MMSI', 'MEDX', 'DORM', 'IMVT', 'AXSM', 'PRDO', 'MORN', 'AXGT', 'SSB', 'EDIT', 'DOYU', 'SSCCQ', 'EDMC', 'MOR', 'DOOO', 'MOMO', 'BHF', 'SRNE', 'AWAY1', 'DPTRQ', 'TERP', 'SPPI', 'BAANF', 'SRNA1', 'AZPN', 'SPOT1', 'BGCP', 'AY', 'TBL', 'SSRM', 'INSM', 'INSS', 'INSY', 'DPL', 'AVXS', 'ASDV', 'SPLT1', 'MIK1', 'DNEX', 'IMPV', 'DNDNQ', 'MIK', 'SYUT', 'PRGS', 'EFNT', 'EGBN', 'AVEI', 'TGNT', 'PRGX', 'PRGY', 'MIDD', 'SSP', 'AVID', 'TFM', 'BIGC', 'EEFT', 'AVT', 'MIME', 'TFSL', 'AVNX', 'INVA', 'EFII', 'BIGT', 'DNLI', 'DNKN', 'AVNR', 'PRGN1', 'DRIV1', 'TERN', 'BFRE1', 'DYAX', 'MMGR', 'BEAV', 'BEC', 'BECN', 'PPRO1', 'INMRY', 'MNTA', 'DWA', 'POWI', 'PRAA', 'PRAH', 'TELL', 'SQST', 'BBRC1', 'BEAT', 'MOGN', 'MNKD', 'BDK', 'TECD', 'PPC', 'DTPI', 'DTRM', 'MNMD', 'PPBI', 'BCPC', 'MNRO', 'PPDI', 'MMYT', 'DURA1', 'BCST1', 'BCSI', 'EAC1', 'SPSNQ', 'TCF', 'BANR', 'BEXP', 'DRYR', 'INET1', 'INRG', 'BARE', 'BANF', 'MOVE', 'BAND', 'DRTN', 'MPWR1', 'INDB', 'TBPH', 'BFPT', 'INOV', 'EBIX', 'INFA', 'INFI', 'TENF', 'SPSC', 'TENB', 'DSGX', 'BBEPQ', 'TCAT', 'MPPP', 'TCBI', 'INFN', 'BBIO', 'INO', 'EAGL1', 'AVCT1', 'BLUD', 'BLSW', 'ENTA', 'ENTG', 'IPXL', 'MRCY', 'ENTU', 'ASWX', 'IRBT', 'MEOH', 'MENT', 'TIGO', 'DHC', 'ENGA1', 'BLMN', 'DIAL1', 'MRBK1', 'PRSF', 'MRCH', 'INGP', 'ENPH', 'MESO', 'ENSG', 'PNK', 'BLUE1', 'ASPX', 'EPAY', 'TLCP', 'SONC', 'EPZM', 'ASIA', 'SOHU', 'MRTX', 'TLCR', 'ASFD', 'BNR', 'ILG', 'ASKJ', 'TKLC', 'SONE', 'PRTA', 'ASMIY', 'EPNY', 'TIVO', 'ASND', 'TINTA', 'PNFP', 'IRDM', 'MRD', 'ASPS', 'ASPT', 'BLKB', 'BLI', 'AVAV', 'IOSP', 'SPIL', 'SYNH', 'IOVA', 'ELNK', 'ELAB', 'ELNK1', 'ELON', 'PNRA', 'ATYT', 'AUB', 'DIGI2', 'EGOV', 'SYNT', 'AUPH', 'EHTH', 'IOM', 'EIDX', 'EIGI', 'TGTX', 'PRNB', 'MGLN', 'SYNC1', 'BLDP', 'MFRM', 'IMCL', 'EMMS', 'ATHR', 'ATHN', 'SYNA', 'BLDR', 'ENERQ', 'DIGL', 'DIOD', 'THQIQ', 'DITC', 'THOR1', 'IMDC1', 'SXCL', 'IPCR', 'ATON', 'MGEE', 'BL', 'DJ', 'IPAR', 'PNP', 'MGI', 'FIVN', 'PXCM', 'HSNI', 'TSEM', 'TSFG', 'HRTX', 'SCOR', 'WEBX', 'KOSP', 'WEBM', 'TSG', 'TSG1', 'SCNT', 'WEN1', 'KOPN', 'WDFC', 'WCRX1', 'WCRX', 'WCII', 'KOD', 'HTLD', 'HTLF', 'WBVN', 'WEN', 'WERN', 'WINS', 'LGTO', 'KRTX', 'SDGR', 'WING', 'WIND', 'HOPE', 'SDC', 'LGIH', 'LGND', 'MCAF', 'LHCG', 'LILA', 'LHSG', 'SCTY1', 'LHSP', 'KRNT', 'WFSI', 'HQY', 'LIHR', 'SCRI', 'WETF', 'WBMD', 'WBD', 'KNSL', 'KEYN', 'IBOC', 'IBTX', 'ICBC', 'TUNE', 'RESP1', 'SBGI', 'RETA', 'RETK', 'ICIX', 'MAXY', 'LITE', 'ICLR', 'TVTY', 'REYN', 'ICON', 'VSEA', 'ICOS', 'ICPT', 'TW2', 'ICST', 'VTS', 'IBKR', 'IBKC', 'SBRA', 'SCHN', 'SCHL', 'HWC', 'RELYQ', 'LIVN', 'TTEC', 'WAFD', 'KLXI', 'HYDL', 'TTEK', 'TTELA', 'SCAI', 'HYSL', 'SBYN', 'KITE', 'VWR', 'MBFI', 'IART', 'KIOR', 'HOMB', 'TRVG', 'HALO', 'SFSK', 'HCSG', 'TREE', 'LAUR1', 'LAUR', 'HDS', 'LCOS', 'RCII', 'WWD', 'HCR1', 'WWCA', 'SFNC', 'WW', 'SFM', 'HELE', 'SFLY', 'SFIX', 'HEROQ', 'RCNCQ', 'SFX.A', 'XCIT', 'TRUP', 'XMSR', 'SHEN', 'XPER', 'HAVSF', 'HAWK', 'TPTX', 'TQNT', 'TRAK', 'XNCR', 'XLRN', 'XCRA', 'SGMS', 'SGMO', 'TRBS', 'SGID', 'LBTE', 'HCBK', 'HCM', 'WTNY', 'SFBS', 'LANC', 'LEXEB', 'WPRT', 'HLNE', 'HLTH1', 'WOOF', 'LEVL1', 'HMHC', 'TRNX1', 'HMIN', 'WMGI', 'WTFC', 'SEEK1', 'HMSY', 'RDUS', 'KTOS', 'REAL', 'LFUS', 'WJCI', 'MCCC', 'KSTN', 'HLIT', 'RDFN', 'HLG', 'KYPH', 'RCOM1', 'LAMR', 'LAF', 'RCPT', 'LEAP2', 'HGSI', 'WSTC1', 'WSTC', 'RDA1', 'HIB', 'WSFS', 'LECO', 'SEPR', 'WSC1', 'HIMX', 'WSC', 'WSBC', 'TRMK', 'HITT', 'VSAT', 'VRUS1', 'ICUI', 'VARI', 'JAMF', 'VC', 'LSTR', 'JACK', 'VATA', 'UMBF', 'LSCC', 'UMPQ', 'RPD', 'LTRPA', 'RMR', 'VA', 'RPAY', 'IWOV', 'RRR', 'SAWS', 'JBLU', 'LPRO', 'JCOM', 'LPSN', 'VERX', 'VERT', 'RSTN1', 'UHAL', 'LQDT', 'RRRR', 'VECO', 'MANT', 'MANH', 'JASO', 'RMBS', 'RSAS', 'IWAV', 'RP', 'ROIC', 'LWSN', 'LX', 'USBC1', 'MACR1', 'UPLCQ', 'LZ', 'RNST', 'URBN', 'UONE', 'ITRU', 'UTSI', 'LTXB', 'UTIW', 'UTHR', 'IVIL', 'LUFK', 'UNEWY', 'USWB', 'UNIT', 'ROSE1', 'ROOM', 'ITXC', 'ITVU', 'USIX', 'ROLL', 'JCOR', 'RTHMQ', 'TXRH', 'RGNX', 'VPHM', 'SAIA', 'TXG', 'SAGE', 'MATW', 'SAFM', 'VNWI', 'UAPH', 'SABR', 'VNRSQ', 'VNOM', 'VMSI', 'VLY', 'TXCCQ', 'SALE', 'LNCR', 'TWST', 'KDNY', 'IDCC', 'VRTA1', 'SATS', 'VRRM', 'VRNT', 'VRNS', 'TWNK', 'VRM', 'SAPE', 'TWOU', 'VRIO', 'RGEN', 'RGLD', 'LNCE', 'LNCOQ', 'UAXSQ', 'VIAN', 'LORL', 'RVBD', 'VIE', 'RUSHA', 'VICR', 'RUN', 'LOXO', 'RLAY', 'UFPI', 'JP1', 'JDEC', 'VG', 'RTLR', 'UGCE', 'LPLA', 'LPNT', 'LOPE', 'RVMD', 'JJSF', 'RXT', 'RXSD', 'VLDR', 'JOINA', 'JOBS', 'VKNG1', 'LOGM', 'JNIC', 'UBSI', 'JMED', 'UCBHQ', 'UCBI', 'LOOK', 'UCOMA', 'VIRT', 'JLG', 'VIR', 'HANAY', 'FCLR', 'AKRXQ', 'FOSL', 'FOXF', 'AKTX', 'FORM', 'AKCA', 'AIXNY', 'PVTB', 'AIRM', 'FRME', 'PWAVQ', 'AINV', 'ALDR', 'FNFG', 'FNGN', 'ALLR', 'ALLO', 'ALLK', 'FNSR', 'ALGXQ', 'ALGT', 'FOCS', 'FOLD', 'ALEC', 'FORE', 'AIMT', 'FRPT', 'AIMC', 'FTRL', 'FTDR', 'FTHL', 'AFFX', 'FTSV', 'AFCI', 'PZZA', 'FUEL', 'FULT', 'AERL', 'AFSI', 'AFYA', 'PWER', 'AHGP', 'FSCO', 'PWRD', 'AGRPA', 'PLMR', 'AGIO', 'AGIL', 'AGCX', 'FSV', 'XSELY', 'PTEL1', 'FFBC', 'PTCT', 'ANAT', 'ANAC', 'ANAB', 'AMWD', 'FFIN', 'PTEN', 'AMSG', 'FELE', 'ALRM', 'FCNCA', 'FCOM2', 'FCOMQ', 'FCS', 'FDML', 'FDMLQ', 'ANDW', 'PSYS', 'FEIC', 'FGEN', 'AMRQQ', 'AMRN', 'AMCX', 'FLIR', 'AMBA', 'AMAG', 'FMBI', 'FHB', 'FMCN', 'FMER', 'FMI', 'ALTR', 'ALTE', 'PTV', 'AMED', 'PTNR', 'FHCC', 'AMRI', 'FIBK', 'AMMD', 'FIRE', 'FIRE1', 'AMLN', 'AMKR', 'FIVE', 'PTLA', 'FIZZ', 'FLA', 'AERI', 'QCOR', 'FUTU', 'GOGO', 'GOLD1', 'AAAP', 'SIR', 'ZU', 'ZSPH', 'GPOR', 'GO', 'SINA', 'GPRO1', 'SILK1', 'ZLTQ', 'SILI', 'ZLAB', 'GOCO', 'SIRF', 'AEIS', 'ABGBY', 'SKIL', 'QTT', 'ABCO', 'GNCI', 'ABCB', 'GNET1', 'GNTX', 'AAXN', 'AAON', 'SITE1', 'QUNR', 'QURE', 'SIRO', 'SIGI', 'YRCW', 'GTLS', 'SHS', 'GTS1', 'RAZF', 'SHOO', 'GWPH', 'XYLN', 'HA', 'HAIN', 'YY', 'GTATQ', 'RARX', 'RAMR', 'SIFY', 'GSHD', 'GSIC', 'ZGNX', 'SIE', 'GSM', 'RARE', 'GSPN1', 'GSTRF', 'ABGX', 'ABII', 'ADIC', 'GBT', 'ADELQ', 'GCTY', 'ACTS', 'ACTRQ', 'ACTG', 'QLIK', 'GEN1', 'SMPL', 'AEGR', 'QEDI', 'FWRD', 'ADVS', 'ADVP', 'QFIN', 'ADTN', 'GAME', 'ADRX', 'ADPT', 'QHGI', 'GBCI', 'GBDC', 'QIWI', 'ACOM1', 'GLYT', 'SLAB', 'ACAS', 'GLBC', 'GLBL', 'GLBL1', 'ACAD', 'GLDN1', 'SKYW', 'GLIBA', 'QTRN', 'SKYF', 'GLNG', 'ABLX', 'ABKFQ', 'ABIZA', 'GKNT', 'QSFT', 'ACCD', 'QLYS', 'ACLI', 'GEVA', 'ACIW', 'ACIA', 'ACHC', 'ACF', 'GHDX', 'SLGN', 'ACDO', 'GIII', 'CYMI', 'UPWK', 'CRZO', 'NTRI', 'NEXC', 'NTRO1', 'OCAS', 'CRWN', 'BWE', 'OCLI', 'CSGS', 'CSOD', 'CSIQ', 'PGEN', 'ONCE', 'CSFL', 'OCNW', 'OCNF', 'NTRA', 'CRUS', 'CACC', 'ONB', 'CACQ', 'CRON', 'PBYI', 'NYMT', 'NETL1', 'CAKE', 'CROX', 'PGNY', 'BWLD', 'NEWP', 'CRTO', 'CRSS', 'CRSP', 'BZUN', 'OLLI', 'CTV1', 'NGHC', 'OLED', 'OMCL', 'CTRE', 'OK', 'NFS', 'OMAB', 'CTLM', 'PAS', 'OMPT', 'PETM', 'NTLI', 'CTYA', 'CSRE', 'PFSW', 'PFPT', 'NTOP', 'NFE', 'CSTE', 'ONDS1', 'CTCM', 'ONEM', 'PBCT', 'CRGN', 'PIOIQ', 'CCOI', 'COCO', 'COBT', 'NUAN', 'PCTY', 'CNXT1', 'CCXI', 'CDCAQ', 'NDN', 'PCVX', 'CNVR', 'COGN', 'COGT1', 'CDEV', 'CONE', 'NUVA', 'COMR', 'NEO', 'COMM', 'COLM', 'CCMP', 'COLE', 'COLB', 'COKE', 'NUFO', 'COHR', 'NUAN1', 'CEDCQ', 'CNET1', 'CNDT', 'CNCX', 'CDX', 'CEI1', 'NTIQ', 'PDCE', 'NTGR', 'CNV', 'CDK', 'CDLX', 'CNTO1', 'PIXR', 'CDWC', 'CNQR', 'CNMD', 'CONN', 'CBST', 'NXTV', 'CASA', 'CARI', 'CPTH', 'CPST', 'PCH', 'PHRM', 'CPNO', 'CASY', 'PCO1', 'CARG', 'PHLY', 'CPHD', 'CREE', 'CALD', 'NXTP', 'CALM', 'CALP', 'CRDN', 'NETE1', 'NXST', 'NXGN', 'CATM', 'CBSS', 'NEOG', 'PCRX', 'CORT', 'CORR1', 'CORE', 'CBPO', 'CBRL', 'CBSH', 'COSN', 'CATP', 'CATY', 'CAVM', 'CBF', 'NEON2', 'NVLS1', 'CMTN', 'PETC', 'OSIP', 'OVER', 'OSIS', 'OUTR', 'OTTR', 'NOVT', 'OSTK', 'NST', 'ORIG', 'OPTV', 'NSCN', 'NSCP', 'ORBK', 'NRPH', 'OXPS', 'OZK', 'PAAS', 'NPNTQ', 'NPLSQ', 'PAHC', 'PAE', 'PACW', 'NPSP', 'NSIT', 'ORTL', 'OPLK', 'ONXX', 'OPCH', 'NICE', 'OPEN', 'OPK', 'PEGI', 'PEGA', 'PENN', 'NMIH', 'NLCS', 'NMRK', 'NTCT', 'PDCO', 'NIU', 'PDLI', 'PLAT1', 'PINC', 'PLAY', 'NCNT', 'NCNO', 'MWIV', 'MYOK', 'CETV', 'CENX', 'MYOG', 'CMOS', 'CMPR', 'CFFN', 'CMRX', 'CEPH', 'CENT', 'MYG1', 'MYGN', 'PLCE', 'PLCM', 'NBLX', 'NBCI', 'NBRV', 'NAVI1', 'NARI', 'NAVI', 'NAVG', 'NATI', 'CKFR', 'CHNG', 'CHDN', 'CLDX', 'CHSI', 'CHTRQ', 'CITC', 'CIGI', 'CMCT', 'CLIC', 'CHAP1', 'CLMT', 'CLRN', 'CLVS', 'CLWR', 'PROS1', 'TPTI', 'UNTD', 'RAMP1', 'PTMN', 'UCFC', 'UNTD1', 'TWPG', 'NWPX', 'TUTS', 'TVIA1', 'UFPT', 'RAYS1', 'PSDO', 'MAXS', 'PTXP', 'MB', 'MAIR', 'QH', 'UFCS', 'PTXTQ', 'TPTH', 'NWLI', 'NVEE', 'TVIAQ', 'MAFB', 'PSNL', 'TRAD', 'MDLI', 'TRCD', 'MAPX', 'TUSK', 'NVCN', 'NXTM', 'NWRE', 'UNCAQ', 'TRCA', 'PRPL', 'PRPL1', 'NSSC', 'PSRC', 'PSPT', 'UNCA', 'RAPT', 'NUZE', 'PRPO', 'QK', 'TRBN', 'TWTI1', 'TRVL', 'NSTA', 'NSPH', 'TUTR', 'RAVN', 'NSPK', 'TRVN', 'MAG1', 'NSPR1', 'UNBJ', 'UNIZ', 'TWRI', 'PTMK', 'PROX', 'UNISQ', 'MDAS', 'QNTA1', 'NVEC', 'TPPHQ', 'MDCA', 'TONEQ', 'NSDA', 'UPHN', 'PRIA', 'UPFC', 'PRIM', 'NXW', 'QADB', 'MAXM', 'TONS1', 'TOPP', 'PRMA', 'MDEA', 'PSMI', 'PTUS', 'RBNC1', 'RBNC', 'TSCN', 'RBMG', 'QCRH', 'TWIN', 'NWK', 'NSCT', 'NVDM', 'RBPAA', 'TWGP', 'PTRY', 'MAXN', 'UPUP', 'TOBC', 'UPSL', 'PTSI', 'TWAV', 'UPRO1', 'UPLD', 'MDIZQ', 'UDFI', 'QCBC', 'TOMO', 'UDRL', 'TOMOY', 'UPCS', 'TSC', 'TOR', 'TPIC', 'PRMX', 'TWLL', 'MDCC', 'TPCG', 'NTMD', 'TPCO', 'NXTR', 'PTIX1', 'RBB', 'NVDQ', 'TRYF1', 'TVLY', 'PROG', 'PROJ', 'NWKC', 'UNXL', 'UNWR', 'MADGQ', 'TVSFF', 'UEPS', 'PRME1', 'UOUT', 'UEIC', 'PTVCB', 'PTVL1', 'PSEM', 'NSIL', 'MDCM', 'RBCN', 'TWLB', 'TSBS', 'MBWM', 'PRMS', 'RBCF1', 'MARK', 'PRMW1', 'RBCAA', 'MDCI', 'TOUR', 'MBVT', 'NTLS', 'UBX', 'MARQ', 'ULAB', 'UIRT', 'PRTLQ', 'PRTM', 'UIHC', 'NVTP', 'TRLG', 'PSTI1', 'TRMAQ', 'NTRT', 'MCFT', 'PRTR', 'MATR1', 'NVMI', 'TRMD', 'TRS', 'NTUS', 'NTSP', 'PRTS', 'NTK', 'UBNK', 'MCEP', 'PXPLY', 'TRIV', 'PVTL1', 'NXTC', 'TYPE', 'TRIN', 'TZIX', 'QUIK', 'NTGX', 'MAPS', 'TTES', 'ULBI', 'MCNS', 'NTWO', 'TZOO', 'PTE', 'MBII', 'TRST', 'QTRX', 'TRIP2', 'PWEI', 'MAPP', 'PT', 'PTILY', 'PSTX', 'PRWK', 'MATR', 'MATK', 'PRVD', 'QNST', 'MCKC', 'NTKK', 'PXP1', 'NXCD', 'UGNEQ', 'QRSI', 'MCHX', 'PRVL', 'MBNY', 'NXEO', 'QRHC', 'QNTS', 'MBOT', 'UBID1', 'NTKI', 'TST', 'QTNT', 'PSUNQ', 'TRMPQ', 'TTCF', 'PSTV', 'MCGC', 'TRPH', 'TSYS', 'MBIN', 'QTNA', 'TSTRQ', 'MCLL', 'MBIO', 'PRVB', 'NTSL', 'TSRX', 'QMAR', 'NTK1', 'QMDC', 'MCLD', 'NTLA', 'TTGT', 'MBRN', 'PTEC', 'PRRC', 'NURO', 'PTI', 'PUYI', 'PVAC', 'RADN', 'MCSI', 'NWBI', 'NSTG', 'TREE1', 'PSSI', 'NUHC', 'NSTR', 'NTBKQ', 'QXMCF', 'TTPH', 'MATX1', 'NTCR', 'UBNK1', 'MCRI', 'PRSC', 'MAKO', 'ULCM', 'TTXGQ', 'NTLO', 'RAIN', 'RAIL', 'TXCOQ', 'PRQR', 'TRCR', 'NUTR', 'TRDO', 'MBUU', 'PXLW', 'MCBC', 'PUMP1', 'TUDO', 'TRDRF', 'PYPL1', 'NSTC', 'RADS', 'MCSW', 'TUBE', 'MAKR', 'MBTF', 'PRSE', 'TSIX', 'TRHC', 'NTPA', 'TTMI', 'TXUI', 'TTLA', 'TRIB', 'NTEC1', 'PVSW', 'NVTR', 'MCOM', 'PRTH', 'MBHIQ', 'ULH', 'TYHT', 'TYME', 'PRTK', 'NTGN', 'TRIDQ', 'MCOAQ', 'ULTEQ', 'PRSW', 'QUMU', 'TTOO', 'PRSS', 'TXMD', 'NVIV', 'PTGX', 'QLTY', 'MCBS', 'TTPA', 'NUCO', 'NVLNF', 'MCOX', 'TRFI', 'TTNP', 'PRXIQ', 'NVUS', 'PSTB', 'PRST', 'TRUE', 'MCRB', 'UCTT', 'SMOD1', 'RCBK', 'SEEL', 'SERO', 'STB', 'STAT', 'SFAM', 'NBEV', 'NBBC', 'SSTI', 'NBAK', 'SFFB', 'SSPK', 'NAUT', 'SSNC1', 'SFNT', 'NATR', 'SFST', 'MOND', 'NATL', 'MONE', 'MONIE', 'STBA', 'MOFN', 'MOFG', 'NCBS', 'NCMI', 'STGC', 'NCIT', 'STFC', 'MNTG', 'STEL1', 'SELB', 'STEL', 'STEC1', 'SEQU1', 'MOBI', 'SEMI', 'MOBI1', 'SENEA', 'MOBL', 'NBTB', 'STBZ', 'MODL', 'NATH', 'MONT', 'SFUN1', 'SRCE', 'NAFC', 'MOTV', 'SGNT', 'SGNT1', 'SGRY', 'SGSF', 'SGTL', 'MZOR', 'MOVIQ', 'SRCP', 'SQSW', 'SGYP', 'MYRG', 'SQNM', 'MYPT', 'SQI', 'SHF', 'SHFL', 'SGNL', 'SRDX', 'MORF', 'SRRA', 'SSBX', 'SSAX', 'SSAG', 'SFXE', 'SRVY', 'SG', 'SRSC', 'SRRK', 'SGA', 'SREV', 'NASC', 'SGC', 'NAPS', 'SGH', 'MOSY', 'SGI', 'SRGA', 'SGICQ', 'STII', 'NCMV', 'MJRC', 'STIM', 'SUPR', 'NDLS', 'SCFS', 'SUPC', 'MLAB', 'MLAN', 'MLND', 'SCHSQ', 'MLNTQ', 'SUNH', 'SCKT', 'NCTY', 'SCLN', 'SCMP', 'MLTC1', 'SCNB', 'MLTX', 'SUGN', 'MM1', 'SUPR1', 'SUPX', 'SURE1', 'SVGI', 'NEOF', 'SBSE', 'SBSI', 'SBT', 'SVRA', 'SBTK', 'SBX', 'SVNTQ', 'MKTG', 'SURF', 'MKTO', 'SVA', 'SCAC', 'MKTW', 'SCAI1', 'SURW', 'NEI', 'NDRM', 'MMCN', 'NCSM', 'STXS', 'STOK', 'STRO', 'NCOG', 'MMPT', 'STRL', 'NCNXQ', 'SDRC', 'SDTHQ', 'SDXC', 'SEAC', 'MMLP', 'STNR', 'STNL', 'SECD', 'SECO', 'STML', 'NCNA', 'MNOV', 'STKL', 'NCOM', 'MMGR1', 'STXB', 'SCRX', 'SCON', 'NCSAA', 'STUA', 'SCOP2', 'STTX1', 'NCRX', 'SCOXQ', 'SCPL', 'STST', 'STRT', 'SCSC', 'SCTC1', 'SCTT', 'STSA', 'SCUR', 'SCVL', 'STRX', 'SCWX', 'SQBK', 'SQBG', 'SHG1', 'MPAA', 'SLNK', 'MTNT1', 'SLOT1', 'SNRS1', 'SLP', 'SNRR1', 'SLRC', 'SLRX', 'SLS', 'MSCP', 'SNNA', 'SNMX', 'MSCS', 'MTLS', 'MTLM', 'SNIC', 'MSEL', 'MSEX', 'MTLG', 'MTNT2', 'SNSS', 'MSCA1', 'MTP', 'MROI', 'MRSN', 'MRTN', 'MTSC', 'MTRX', 'MRUS', 'SLDB', 'SLFI', 'SLGL', 'SNTL', 'MRVT', 'MSBC', 'SOAP', 'SNWL', 'MSBI', 'SNTS', 'MSBK', 'MTOH', 'SNEX', 'MSFG', 'SLXA', 'SMSI', 'SMTL', 'SMDI', 'SMTK', 'SMDK', 'MTEX', 'MTEM', 'SMT1', 'SMT', 'SMSC', 'SMTX', 'SMMC', 'SMRA', 'SMMF', 'SMMT', 'MTCH1', 'SMMX', 'MTCT', 'SMNS', 'SMTS1', 'MSTX', 'MSGI', 'SNC', 'MTIC', 'SNDX', 'MSON', 'SMBC', 'SMBI', 'SND', 'SNCA', 'SMBK', 'MTGNY', 'MSSR', 'SMCI', 'MSPD', 'MSPR', 'SNBR', 'MTGE', 'MSSN', 'SNBI', 'SNBC', 'MRNS', 'SOMX', 'SONA', 'SIEN', 'SIBN', 'SIC', 'SPRI', 'SPRD', 'MXIP', 'SIEB', 'MQST', 'SPOK', 'MRBA', 'MXWL', 'SIGA', 'SPNS', 'MXICY', 'SPNE', 'SIGM', 'SPNC', 'SILC', 'SILK', 'MPVD', 'SPRT', 'SPLX1', 'SHOP1', 'SPYG1', 'SPWY', 'SHLM', 'SPWX', 'SHLOQ', 'SHLR', 'SPWH', 'SPTN', 'SHOR', 'SHYF', 'MPLX1', 'SHOS', 'SHRP', 'SPT', 'SPSX', 'SPSS', 'SHVA', 'MYCO', 'SPMD1', 'SPLT', 'SONG', 'MRIN', 'MVBI', 'MRII', 'SP1', 'SP', 'MTWV', 'SKIL1', 'SKP', 'MTSN', 'SORCQ', 'SITM', 'SKUL', 'MRKR', 'SKYS', 'SKYT', 'MRLL', 'SONO1', 'SONO', 'MRLN', 'SIXD', 'MVIS', 'MWWW', 'MWHS', 'SIMG', 'MWP', 'SPLN', 'SPLI', 'SPLH', 'SIMO', 'SPKE', 'SPI', 'SPGLQ', 'SISB', 'SINT1', 'SPFI', 'SIPX', 'SPCT', 'SIR1', 'SPBC1', 'MRGE1', 'SIRT', 'MKD', 'MJCO', 'RCCC', 'NODK', 'MEMP', 'MEND', 'NMRD', 'THFF', 'THER', 'RJETQ', 'NMFC', 'THDO', 'THCB', 'NLTX', 'RLMD', 'RLOC', 'THBR', 'TH', 'RLRN', 'RLYP', 'NKTX', 'RMDY', 'NKSH', 'RIOT', 'THOR', 'RINO1', 'RGX', 'TILE', 'NOBLQ', 'MEET', 'MEIP', 'TIGR', 'RGP', 'TIG2', 'TIG', 'NNBR', 'RINO', 'NMTR', 'RIBT', 'RICK', 'NMTC', 'THRT', 'THRM', 'RIGL', 'RILY', 'TGRP', 'RMIXQ', 'RMNI', 'NIKU', 'NITE', 'NINE1', 'MESG', 'RNOW', 'MESG1', 'TEXP', 'NILE', 'RNVS', 'TEUM', 'MESA', 'ROAD', 'TESS', 'ROAD1', 'NIHD', 'TESO', 'METC', 'ROCK', 'TERA', 'MESAQ', 'RNET', 'RMOC', 'RNA1', 'NKBP', 'TGLS', 'RMTI', 'TGLO', 'RMTR', 'RMY.A', 'RNA', 'TGL', 'TGICQ', 'MERU', 'MERC', 'TGA', 'RNA2', 'RNAI', 'RNBO', 'TFSM', 'NK', 'RNDC', 'RGLS', 'TIMHY', 'SWAV', 'RGCIQ', 'MDST', 'RDNT', 'RDRTQ', 'MDTH', 'NPTT', 'RDWR', 'TMRK', 'MDTL', 'MEAD', 'REDU', 'NPSI', 'TMDX', 'MEAS', 'MECAQ', 'REFR', 'MECK', 'TMARQ', 'TLXN', 'REGI', 'RDI', 'NRC', 'NREB', 'RCNI', 'RCCK', 'TNSI', 'RCKT', 'TNI', 'TNGO', 'RCM', 'NRIX', 'TNFI', 'NRGN', 'RDEN', 'RCOT', 'RCPIQ', 'MDSI', 'TNAV', 'RCRC', 'RDA', 'NRES', 'RDEA', 'REGI1', 'TLVTF', 'TLTN1', 'NOVA1', 'NOVN1', 'RESC', 'NOVN', 'TLCM', 'MEDQ', 'TKTX', 'RETO', 'TKTM', 'MEDT1', 'TLCT', 'TJCO', 'TIXX', 'TIVO1', 'TITN', 'TIPT', 'REXI', 'REXX', 'NOPT', 'REPL', 'REPH', 'TLTN', 'REMC', 'TLSP', 'MEDC', 'NPIX', 'NPCI', 'RELY', 'TLRK', 'MEDE1', 'TLMR', 'TLMDA', 'REPB', 'TLGT', 'NPBC', 'TLGD', 'TLGB', 'TLEO', 'REMY', 'TLCVF', 'RENT', 'ROHIQ', 'TEMPA', 'METG', 'METR', 'SAAS', 'SYRS', 'SABA', 'SYPR', 'NETZ', 'SYNQ', 'SAFT', 'SAGI', 'SYNP', 'SYNO', 'SYNM', 'NETO', 'NETM', 'SYNC', 'SALM', 'SYMM', 'MIKL', 'SYKE', 'SYGR', 'RYTM', 'RYAN', 'RXDX', 'TALR1', 'RVLT', 'RVNC', 'TANN', 'MIAM', 'MICA', 'RVSIQ', 'TALX', 'RVSN', 'NEWT', 'SZMK', 'TALK1', 'RWDT', 'TAGS1', 'TACT', 'MICU', 'TACO', 'NEWS', 'TA', 'NETG', 'SAMC', 'SYBX', 'SBIT', 'SBCF', 'MIST', 'SWIR', 'NEOS', 'NEON1', 'MITI1', 'MITK', 'SBIB', 'SBKC', 'MISI', 'MITL', 'SBLK', 'MITO', 'SWBI', 'SBLK1', 'SBLUQ', 'SWBD', 'SBMC', 'SBBP', 'MIRM', 'SANG', 'SAVA', 'SYBT', 'NETC2', 'SASR', 'SY', 'SATCQ', 'MIND', 'SATH', 'SWWC', 'NESR', 'SWSH', 'SAVLY', 'SWW', 'NERV', 'SBAS', 'MIPIQ', 'NEPT', 'SWTX', 'SWSI', 'RVFD', 'NEXL', 'NEXT', 'RPXC', 'NGM', 'MFSF', 'RPRX1', 'MGAM', 'TCMD', 'RPTP', 'RPTR', 'RPTX', 'RRBI', 'NGPS', 'RRGB', 'MGEN1', 'RRM', 'TCDA', 'MGIC', 'MGL', 'TCBK', 'RSCR', 'TCPC', 'TCRR', 'RSLCF', 'MFH', 'NHWK', 'TELM', 'NHTC', 'METX', 'TEDU', 'SMOD', 'TEAR', 'ROMA', 'MFIN', 'TCTY', 'TDFX', 'ROSE2', 'ROSEQ', 'NH', 'MFLX', 'ROSI', 'ROWE', 'TCX', 'NGEN', 'RSLN', 'TAOP', 'NFBK', 'TBCC', 'RTSX', 'TBBK', 'NFIN', 'MGTA', 'RUBY', 'RUE', 'RUHN', 'MGTX', 'TBFC', 'RURL', 'TAYC', 'RUTH', 'NEXT1', 'TAST', 'TARRQ', 'MHGC', 'MHLD', 'RTRX', 'NFLDQ', 'RSNT', 'TBNK', 'RSTA', 'RSTI', 'TC', 'MGNX', 'TBSIQ', 'MGPI', 'TBRA', 'MGR1', 'RSTO', 'MGRM', 'MGRC', 'RSTR', 'RSYS', 'NFO1', 'TBK', 'RTKHQ', 'RTLX', 'TBIO', 'TECU', 'AFOP', 'URGN', 'CNTG', 'HBIX', 'HBMD', 'COGT', 'HBNC', 'CODX', 'COBZ', 'HBT', 'COB', 'HCAC', 'CNXS', 'HCAT', 'CNXN', 'CNVR1', 'HCCI', 'HCCO', 'GRDG', 'CNST', 'HCKT', 'CNSL', 'HCOM', 'CNOB', 'HCT', 'HDLD', 'HDNG', 'HDP', 'CNDS', 'CNCT1', 'HDSN', 'CNCE', 'CNBKA', 'COHU', 'COLL', 'COLY', 'COMM1', 'CPICQ', 'CPHRF', 'CPHL', 'HAKI', 'HALL', 'HARB', 'HARP', 'HARS', 'CPAA1', 'CPAA', 'COWN', 'COVS', 'COSIQ', 'CORV', 'CORSQ', 'CORL', 'CORI1', 'CORI', 'HAWKQ', 'HAYN', 'HAYZQ', 'COOP', 'COOL1', 'HBCP', 'CONV', 'CONR', 'CONEQ', 'CONC1', 'COMV', 'HEAR', 'HEARZ', 'HEC', 'HHR', 'CLSCQ', 'CLRT', 'CLRK', 'HIBB', 'HIC', 'HIFN', 'CLNE', 'CLMS', 'CLLS', 'CLKS', 'HILL', 'CLFY', 'CLFD', 'CLFC', 'CLDA', 'CLCX', 'CLCT', 'CLCD', 'HITK', 'CLBK1', 'CLBK', 'HITS', 'CLAY', 'CLAR', 'CLAC', 'CKXE', 'CKSW', 'CKSG', 'CKEC', 'CLSD', 'CLSR', 'CMTY', 'CLTR2', 'CMTO', 'CMTL', 'CMTA', 'CMSX', 'HEES', 'CMSB1', 'HELX1', 'CMPX1', 'HEOP', 'CMPC', 'CMNT', 'CMLSQ', 'CMLS', 'HEVVQ', 'CMIKA', 'CMGE', 'HFFG', 'CMEDQ', 'HFWA', 'CMDCQ', 'CMCO', 'HGEN', 'CMCAF', 'CMBM', 'HGIC', 'CLZR', 'HGSH', 'CLXT', 'CLTX', 'CPII', 'CPIX', 'CPKI', 'CTRN', 'CTRCQ', 'CTRA2', 'CTMX', 'CTMI', 'CTIX', 'CTIC', 'CTHR', 'CTG', 'GSKY', 'CTESY', 'CTEC', 'CTEA', 'CTCT', 'CTCO', 'CTCI', 'CTBI', 'GSM1', 'CTAL', 'CTAC1', 'GSMG', 'CSWI', 'CSWC', 'CSUNY', 'CSTR', 'GSNX', 'GSOF', 'CSTL', 'GSOL', 'GSTN', 'CTRL', 'CTRX1', 'GSUM', 'GSBC', 'CVON', 'CVNS', 'GREY', 'CVLG', 'CVGW', 'CVGP', 'CVGI', 'CVCY', 'GRIC', 'GRIF', 'CVAS1', 'CUTR', 'CUSM', 'GRNB1', 'GRNQ', 'CUREQ', 'CUNO', 'CUNB', 'GROW', 'CUE', 'CUBE1', 'CTZN1', 'GRTS', 'CTWS', 'GRTX', 'GRVY', 'CTST', 'CTSO', 'GRWG', 'GSTX1', 'CSKI', 'CPLA', 'CRMT', 'GVHR', 'CRIS', 'CRIO', 'CRIC', 'GW', 'GWGH', 'CRESY', 'CREL', 'CREG', 'GXDX', 'CRDT1', 'GXGX', 'CRCL1', 'CRBP', 'CRAY', 'CRAI', 'CRA', 'CPXX', 'GYMB', 'CPWM', 'CPTS', 'HABK', 'HABT', 'CPSL', 'CPSI', 'CPRX', 'HAFC', 'CPRK', 'CPLP', 'GURE', 'CRNC', 'CSII', 'GUID', 'GTCB', 'GTHR1', 'GTHX', 'CSCD', 'CSBK', 'CSBI1', 'GTIV', 'CSARQ', 'CRYP', 'GTOPQ', 'CRXLY', 'CRXA', 'GTRC', 'CRVS', 'CRVL', 'CRTX1', 'CRTX', 'CRTO1', 'CRTN', 'CRTK', 'GTTLB', 'GTYH', 'CRSA', 'CRRSQ', 'CRRC', 'GUCO', 'CROS', 'CRNX', 'CRNT', 'CKCM', 'HLIO', 'CJHBQ', 'CBIS1', 'HYSW', 'CBEY', 'CBEH', 'HZWV', 'IAAI', 'CBBI', 'CBAY', 'CBAT', 'IACH', 'IACT', 'IAIA', 'IAWK', 'CATT', 'IBAS', 'CATC', 'IBCP', 'CAST1', 'CASS', 'IBIS', 'CASI', 'CASH', 'CASC', 'IBNKQ', 'CARS2', 'CARO', 'CARB', 'CARA', 'ICCI', 'CAPA1', 'CBIO', 'CBKN', 'CAN', 'HYMC', 'CCMC', 'CCLP', 'HUMC', 'CCIX', 'CCIL', 'CCGM', 'HURC', 'CCCG', 'HURN', 'HUSN', 'CCBN', 'CCBL', 'CCBI', 'CCBG', 'HWCC', 'CCAP', 'CBXC', 'CBTX', 'HWKN', 'CBOU', 'CBONQ', 'HX', 'HYAC', 'HYAC1', 'HYBR', 'CBMI', 'CBMG', 'HYGS', 'CBLK', 'CANI', 'CAMZ', 'HUIZ', 'BWMX', 'IDBE', 'BWEN', 'BWEBF', 'BWB', 'BWAY1', 'IDEV1', 'BVEW', 'IDEX', 'BV1', 'BUYX', 'BUSE', 'IDNX', 'IDRA', 'BUCA', 'IDWK', 'BTRX', 'IDXC', 'BTOB', 'IDXG', 'IDYA', 'BTFG', 'BTAQ', 'BTAI', 'IESC', 'BSTE', 'BSTC', 'BSRR', 'BSQR', 'IFCI1', 'ICXT', 'ICUB', 'CAMT', 'ICTG', 'CAMP', 'ICFI', 'CAMD', 'ICGX', 'ICHR', 'CALL', 'ICII', 'CALCQ', 'ICLK', 'CALA', 'CAIS', 'CAIR', 'CAGC', 'CAFD', 'CADX', 'ICMI', 'CACS', 'ICOC', 'CACB', 'CAC', 'CABA', 'CAAS', 'ICOR1', 'BYSI', 'BYND1', 'ICPT1', 'ICSL', 'ICST1', 'BXM', 'CCN', 'CCNE', 'HLND', 'HMTT', 'CHIC1', 'HNBC', 'CHFN', 'HNCS', 'HNH', 'HNRG', 'HNSN', 'CHEF', 'CHDX', 'CHCO', 'CHCI', 'CHBT', 'HNVD', 'HOFF', 'HOFT', 'HOGS', 'CGRB', 'CGPI1', 'CGLD2', 'HOL', 'HOLI', 'HOLL', 'CGEN', 'CGCP', 'HOMG', 'CGBD', 'CFWY', 'HONE', 'HOOK', 'HMTV', 'CHMA', 'CFSG', 'HMST', 'CIVB', 'HLP', 'CITP', 'HLSSF', 'HLYS', 'CIMA', 'CIIC1', 'CIIC', 'CIH', 'HLYW', 'CIFC', 'HMCH', 'CHZS', 'CHYR', 'CHUY', 'CHUX', 'CHUBA', 'CHTT', 'CHTP', 'CHRX', 'CHRS1', 'CHRS', 'CHRM', 'HMIN1', 'CHRD', 'CHPM', 'CHPC1', 'CHNR', 'HMSK', 'CFSI', 'HORC', 'HUGH', 'HRZB', 'CEDU', 'CECE', 'CDZI', 'CDXS', 'CDXC', 'HSII', 'CDTS', 'HSKA', 'HSON', 'HSTM', 'CDRBQ', 'CDNW', 'CDNA', 'HSTR', 'CDMO', 'HTBI', 'HTBK', 'CDIC', 'HTBX', 'HTCH', 'HTHR', 'HTRN', 'CCSC1', 'HTWR', 'CCRN', 'HUBG', 'CCRD', 'CCRC', 'CCOW', 'CEGE', 'CELH', 'CFNL', 'CELL', 'HOTJ', 'CFMS', 'HOTT', 'HPGP', 'CFFA', 'CFCP', 'HPK', 'CFBX', 'HPLA', 'HPOL', 'HPOW', 'HPRT', 'CFB', 'HPTX', 'CEXP1', 'CEVA', 'CETC', 'CERTF', 'CERS', 'CERO', 'HRBC', 'CERE', 'HRBN', 'HRBT', 'HRLY', 'CENF', 'HRMN1', 'CEMP', 'CELSD', 'CVTX', 'CVVT', 'URGP', 'FTD', 'EPIQ', 'FSTR', 'EPIC', 'EPHC', 'EPEXQ', 'EPAX', 'EOPN', 'EOLS', 'ENZY', 'ENWV', 'ENVY', 'FSYS', 'FTAC', 'ENVE', 'ENTR1', 'GRBK', 'FTD1', 'ENTN', 'ENTL', 'FTDI', 'FTEK', 'ENSI', 'FTEO', 'FTFC', 'ENOC', 'FTGX', 'ENMC', 'ENLV1', 'FTLK', 'ENG', 'EPIX', 'EPIX1', 'EPOC', 'EPRE', 'ESND', 'ESMK', 'ESLRQ', 'ESIO', 'FSBK', 'ESIC', 'ESHR', 'ESHEQ', 'ESFT', 'ESEA', 'FSCI', 'ESCH', 'FSCT', 'ESBF', 'ERYP', 'ERT', 'ERS', 'EROC', 'FSGI', 'ERII', 'FSII', 'ERES', 'FSIN', 'FSLA', 'FSNMQ', 'FSNN', 'EQBK', 'FSRV', 'EPRS1', 'ENDO1', 'ENCY', 'ENCD', 'EIGR', 'EH', 'EGT', 'EGRX', 'EGRT', 'EGNI', 'EGLTQ', 'EGLT1', 'EGLS', 'EGLO', 'FXEN', 'EGLE', 'EGHDQ', 'EGGS1', 'EGFC', 'GABC', 'EGC', 'EGAN', 'GAGA', 'EFTD', 'EFSC', 'GAIA', 'GAIN', 'GALT', 'GALT1', 'EFAX1', 'EF', 'EESI1', 'EELN', 'GASS', 'EICU', 'FWP', 'ENBX', 'FWNC', 'FTUSQ', 'EMUS', 'FTWRQ', 'FULC', 'EMKR', 'EMEX', 'FUQI', 'EMCI', 'FURX', 'EMBT', 'FUSN', 'EMAG1', 'FVCCQ', 'FVE', 'ELRC', 'ELOX', 'ELOT', 'ELOS', 'ELOQ1', 'ELOQ', 'ELNT', 'FWBI', 'ELMG', 'ELIX', 'ELGXQ', 'ELCO', 'ELBO', 'FWMHQ', 'ELAS', 'FSAC', 'ESPI1', 'ESPR1', 'FMAX', 'FFBA', 'FESX', 'FMBK', 'FMBN', 'FEET', 'FEED', 'FDUS', 'FMNB', 'FMTX', 'FMXLQ', 'FNBCQ', 'FCTR1', 'FCSX', 'FCST', 'FCRD', 'FNCB', 'FNCX', 'FCLX', 'FCGI', 'FNHC', 'FNIS', 'FCBP', 'FNKO', 'FCBIB', 'FNLC', 'FCBC', 'FBRX', 'FBRC', 'FBNK1', 'FMBH', 'FFCH', 'FBMS', 'FMAO', 'FISI', 'FINL', 'FINE', 'FINB', 'FLAG1', 'FLAS', 'FICC', 'FIBR1', 'FGXI', 'FGHLQ', 'FLDM', 'FGCI', 'FLDR1', 'FFWM', 'FLFC', 'FLGT', 'FLI', 'FLIC', 'FLMN', 'FLNT', 'FLOW1', 'FFKT', 'FFIC', 'FLSH', 'FLWS', 'FLXN', 'FFFL', 'FLXS', 'FLYI', 'FBNC', 'FOBB', 'FRTZ', 'EVVV', 'FRBK', 'FRCMQ', 'FRDMQ', 'FREDQ', 'FREE', 'FREQ', 'EVLV1', 'EVLO', 'FRG', 'EVGN', 'FRGBQ', 'EVER', 'FRGI', 'FRGO', 'EVAC', 'EURX', 'FRHC', 'EUBK', 'FRNTQ', 'FRP', 'FRPH', 'ETNB', 'ETELY', 'FRPT1', 'FRSH', 'ESTA', 'ESST', 'FRSX', 'FRTA', 'FRAN', 'FPRX', 'FBIO', 'EXAC', 'FODG', 'FOGO', 'FOMX', 'FATB', 'FORR', 'FARO', 'FARM', 'FORTY', 'FANH', 'FOUR2', 'FADV', 'FACT', 'FACO', 'EZPW', 'EZCH', 'EYET', 'FOXH', 'EYES', 'FPAMQ', 'EXTN1', 'EXPI', 'EXPC', 'EXLT', 'EXLN', 'FPFCQ', 'EXEE', 'FPFX', 'FPIC', 'EXAP', 'GBBK', 'EEEE1', 'EEEE', 'GMGC', 'GMLP', 'DGIT', 'DGIN', 'DGII', 'DGICA', 'GMRRQ', 'DFRG', 'GMTA1', 'DFIB', 'GNBC', 'GNCA', 'GNCMA', 'DEVN', 'GNFT', 'DEST1', 'DEST', 'DESC1', 'DERM', 'GNLN', 'GNMK', 'DENN', 'GNOM1', 'GNOM2', 'DEIX', 'DEER', 'DEEPQ', 'GNSC', 'GNSL', 'DEBS', 'GMHI', 'GMET', 'DDSSF', 'DHIL', 'DMAN', 'GLGSQ', 'DLTK', 'DLTH', 'DLLR', 'DLIA2', 'DLIA1', 'DLIA', 'GLLC', 'DKHR', 'DJO', 'DJCO', 'DIVX', 'GLRE', 'GLUU', 'DINE', 'DIMD', 'DIIG', 'GLYC', 'DIGM', 'DIGI1', 'GLYN', 'DIGE', 'DICE', 'DIAN', 'DHXM', 'GMAN1', 'DHOM', 'GMDA', 'DDUP', 'DDMX1', 'GLDD', 'GOVX', 'CYTK', 'CYSV', 'CYRX', 'CYRN', 'CYPB', 'CYOU', 'CYNO', 'GP', 'CYLK', 'CYGN1', 'GPCBY', 'GPFI', 'CYCO', 'CYCN', 'CYCH', 'CYBX', 'CYBS', 'CYBR1', 'CYBE', 'GPP', 'CYAD', 'GPRE', 'CXDO', 'CXDC', 'CWONQ', 'GPSI', 'CWLD', 'CWCO', 'GRAN', 'CYTO', 'CZNC', 'GNSS1', 'GOSS', 'DDIM', 'DDICQ', 'GNTA', 'DDDP', 'DDDC', 'DCTM', 'DCTI1', 'DCTH1', 'GNTY', 'DCOM', 'DCNT', 'GNUS', 'DCGNQ', 'GNVC', 'DBVT', 'DBTK', 'DBT1', 'GOAL', 'GOGL', 'DATE', 'DATC', 'GOMO', 'GOOD', 'DAOU', 'DANKY', 'DALN', 'DAKT', 'GOSHA', 'DAEG', 'GLFD', 'DMLP', 'GBCB', 'EBNX', 'EBIO1', 'EBHIQ', 'EASY', 'EASI1', 'GEDU', 'EAII', 'EAGLU1', 'EAGL2', 'GEHL', 'GELX', 'DZTK', 'DZSI', 'DYT', 'GENC', 'GENE', 'GENI', 'GENTY', 'GENUQ', 'GENXY', 'DXPE', 'DXMMQ', 'DXLG', 'DWSN1', 'GEOC', 'DWRX', 'GEOI', 'GEOS', 'GEOY', 'DVINQ', 'EBKR', 'EBSB', 'DUSA1', 'EBSC', 'EDSN', 'GBIO', 'GBLI', 'GBND', 'GBNK', 'EDGW', 'EDFY1', 'EDEN1', 'GBTB', 'ECYT', 'GCBC', 'ECST', 'GCOM', 'ECPG', 'ECOR', 'GCOR1', 'ECOL', 'GCTI', 'ECLP', 'ECLG', 'ECIL', 'GDEN', 'GDEN1', 'ECHO', 'GDYN', 'GDYS', 'EBTN', 'EBTC', 'GEAC', 'DVAX', 'GERN', 'DMND', 'DRRAQ', 'DRNA', 'DRMD', 'GIFT', 'GIGM', 'DRCT', 'GILT', 'DPRC', 'DPMI', 'DPHC', 'GISX', 'GIVN', 'DOVP', 'DOVA', 'GK', 'DOSE', 'GKIS', 'DOMO', 'GLAD', 'DOCI', 'GLBN', 'GLBR', 'DNET', 'DNCR', 'DNBK', 'GLCH', 'DMTX', 'GLDB', 'DMRC1', 'DMRC', 'GIFI', 'DRRX', 'DUO', 'DRTE', 'GETI', 'GEVO', 'DTSI', 'DTLN', 'DTLK', 'DTIL', 'DTIIQ', 'DTHK', 'DTEC1', 'DTEA', 'DTAS', 'GFN', 'DSTM', 'DSPG', 'DSLGF', 'DSKY', 'DSKE', 'GHCI', 'DSET', 'GHIV', 'DSCP', 'DSCM', 'DSCI', 'GHVIQ', 'DRWI', 'GIBG', 'DRTX', 'DRTT', 'DRTK', 'BSNX', 'IFCJ', 'BSML1', 'LABS1', 'KYTH', 'KZ', 'ZGEN', 'ZEUS', 'ZEAL', 'ZE', 'ZAZA', 'ZARLF', 'KZR', 'ZAIS', 'ZAGG', 'LABL', 'YTRA', 'YTEN', 'YSTM', 'BSFT', 'YORW', 'YONG', 'YMAB', 'YJ', 'YIN', 'YI', 'LAND', 'YDNT', 'YDLE', 'YDKN', 'YBTVQ', 'YANB', 'LASR', 'XYBRQ', 'ZIGO', 'ZILA', 'ZILG', 'ZINCQ', 'KSWS', 'AAII', 'AAI', 'AACQ', 'AACE', 'AACC', 'AACB', 'ZYXI', 'KTII', 'KTO', 'ZVO', 'ZUMZ', 'KTTY', 'ZRAN', 'KTWO', 'ZOOX', 'KURA', 'ZOMX', 'ZOLT', 'ZOLL', 'ZNTL', 'KVHI', 'ZMBA', 'ZLG', 'ZIXI', 'ZIPR', 'ZIP', 'ZIOP', 'KYAK', 'XXIA', 'LATD', 'XTND', 'XENT', 'XELA', 'LCAV', 'XCED', 'LCCI', 'XBKS', 'XBIT', 'WYPT', 'WWWW1', 'WWR', 'WWIN1', 'WWIN', 'WVWCQ', 'WVE', 'WVCMY', 'WTSL', 'WTRH', 'WTRE', 'LCUT', 'LDIG', 'LDIS', 'WTEL', 'LDRH', 'WTBA', 'LDSH', 'LE', 'WSTL', 'LEAF1', 'WSII', 'WSG', 'XENE', 'XERS', 'LAVA', 'LCA', 'LAWS', 'XSPA', 'XRTX', 'LAYN', 'XRIT', 'LBAI', 'XPRT', 'XPRSA', 'LBC', 'XPEL', 'XPDR', 'LBFC', 'XOOM', 'XONE', 'XOMD', 'XOMA', 'XOHO', 'XNPT', 'XNET', 'XMIT', 'XMCM', 'XLTC', 'XLSW', 'XIRC', 'XINGF', 'XIDEQ', 'XICO', 'XGN', 'XFOR', 'AAOI', 'AATI', 'KSPN', 'ADXS', 'ADVM', 'ADUS', 'KFRC', 'KFX', 'KIDS', 'ADSI', 'ADPI', 'ADNC', 'ADMS', 'KIN', 'ADMA', 'ADLTQ', 'ADLR', 'ADFC', 'ADEX', 'ADES', 'ADEP', 'KIRK', 'ADBL', 'KITDQ', 'ADAPQ', 'ADAP', 'ADAC1', 'ACVC', 'ACTV1', 'ACTMQ', 'ACTL1', 'ACTI1', 'ACTI', 'ADVNQ', 'ADZA', 'KLIC', 'KEYW1', 'KALA', 'KALU', 'AGCCQ', 'KALV', 'KANG', 'AGAM', 'AFWY', 'AFM1', 'KBAL', 'KBAY', 'AFIN', 'AFIB', 'KCLI', 'AFCX', 'AFCO1', 'AFAM', 'KE', 'KELL', 'KELYA', 'KERX', 'AERG', 'AEPI', 'AEN', 'AEMI', 'AEIC', 'KEYS1', 'AEHL', 'KEYW', 'AEGN', 'KLDO', 'ACRX', 'KSMT', 'ACAT', 'ACAM', 'KOR', 'KOSN', 'ABUS', 'ABTXQ', 'ABTX', 'KPFS', 'ABSC', 'ABRX1', 'KPTI', 'ABOV', 'ABNK1', 'ABLSQ', 'KRMD', 'ABIO', 'ABEO', 'KRNY', 'ABDR1', 'KROL', 'ABCW', 'ABCR1', 'KRON1', 'KROS', 'ABCD', 'ABBC', 'ABAX', 'KRYS', 'AAWW', 'AAWHQ', 'ACAP', 'KOOP', 'ACRU', 'ACBI', 'ACRTQ', 'ACRS', 'KLXE', 'ACOR', 'ACOM', 'KMAGQ', 'KMDA', 'ACMR1', 'ACMR', 'ACME', 'ACLS', 'KNBT', 'KNDI', 'ACLA', 'ACIU', 'KNDL', 'KNOL', 'ACHN', 'KNSA', 'KNSY', 'ACETQ', 'ACET', 'ACER', 'KNTA', 'KOMG', 'ACCS2', 'ACCL', 'KONA', 'ACCC', 'LEDR', 'LEDS', 'WSBF', 'VKTX', 'VIVO', 'LOIS', 'VITL1', 'VITL', 'LOJN', 'VITC', 'VITA', 'LONG', 'VISN', 'LOOP', 'VIRS1', 'VIRL', 'LOOP1', 'VIOT', 'VIONQ', 'VINA', 'VIMC', 'LOUD', 'LOVE', 'VIAS1', 'VIAS', 'VIAD', 'VIAC1', 'VGNX', 'VGIN', 'LPCN', 'VFSC', 'LPHIQ', 'LPMAF', 'VIXL', 'VLCM', 'VETX', 'VLCT', 'VRCA', 'VRAY', 'LNDC', 'VRAD', 'VRA', 'LNETQ', 'LNTE', 'VOYN', 'VOXX', 'LNTH', 'LNY', 'VOLC', 'VOCS', 'VNWK', 'VNUS', 'VNTV1', 'VNLPY', 'VNET', 'VNDA', 'LOB', 'VMD', 'VLTR', 'VLTC', 'VLSI', 'LOCO', 'VLNK', 'VLNCQ', 'VLGEA', 'LOGC', 'VFF', 'LPTH', 'VREX', 'UTEK', 'USWS', 'USWC', 'LUME', 'LUMO', 'USTB1', 'USSB', 'LUMT', 'USPI1', 'USON', 'USNC', 'USLM', 'LUSA', 'USIT', 'LVCI', 'USFS', 'USFC1', 'LWAY', 'LWIN', 'USCS1', 'USCR', 'LXNT', 'USBE', 'LXRX', 'USAP', 'USAK', 'USAD', 'LYTS', 'UROV', 'MACK', 'UTCI', 'UTMD', 'VERI', 'LTWO', 'LQDA', 'VELTF', 'VEAC', 'VDTH', 'LRMR', 'VCYT', 'VCTR', 'LSBC', 'VCNT', 'VCELA', 'VCEL', 'VCBI', 'VCAM', 'VCAI', 'VBTX', 'LSCP', 'LSON', 'LSPN', 'VBIV', 'VAST', 'VASC', 'VANS', 'LTEK', 'VALU', 'LTRX', 'UXIN', 'UVSP', 'LTVCQ', 'UTVI', 'VRDO', 'VRGE', 'WSBA', 'WINS1', 'WINMQ', 'WINK', 'WINA', 'WIMI', 'WIFI', 'WIBC', 'WHF', 'WGRD', 'LIFC', 'LIFE', 'WGATQ', 'LIFE1', 'WFBI', 'WEYS', 'LINC', 'LIND', 'WEBT', 'WEBS1', 'WEBC1', 'WEB', 'WDRY', 'LINK', 'WCSTF', 'WCRX2', 'WCLX', 'WCBO', 'LION', 'LIOX', 'WBSN', 'WINN', 'WINT', 'LISB', 'WIRE', 'WRTC', 'WRSP', 'LEGH', 'WRLD', 'LEND1', 'WRESQ', 'WRDP', 'LENS1', 'WORK1', 'WOMN1', 'WOLF', 'LEVP', 'WNEB', 'LEXR', 'WMAR', 'WLSC', 'WLNK', 'WLMR', 'WLFC', 'WLDN', 'LFIN1', 'WLBC', 'WLBA', 'WKHS', 'LFMN', 'WJCO', 'LFVN', 'WITS', 'LGCYQ', 'LIPO', 'WAXS', 'VRGY', 'VTEQ', 'LMLP1', 'VTAL', 'VTAE', 'LMNE', 'VSTN1', 'LMNL', 'VSTM', 'VSTH', 'LMNR', 'VSTA1', 'VSTA', 'VSNX', 'LMNS', 'LMNX', 'VSIO', 'LMOS', 'VSEC', 'LMRK', 'VSCN', 'VRTY1', 'VRTU', 'VRTS', 'VRTL', 'VRST1', 'VRSOQ', 'VRSA', 'VRNM', 'VRNA', 'VRLN', 'LMIA', 'LMAT', 'WAVXQ', 'VTIV', 'WAVR', 'WAVO', 'WAVE', 'WATT', 'WATG', 'WASH', 'WARR', 'WANG', 'LIWA', 'WABC', 'VYYO', 'VYTL', 'VYSI', 'VYNE', 'VYGR', 'VXRT', 'LJLB', 'VWRX', 'VVVV', 'LJPC', 'LJPC1', 'LKCO', 'VUSA', 'LKFN', 'LKST', 'LLEN', 'VTRU1', 'VTRO1', 'LLNW', 'AGEN', 'AGFS', 'AGII1', 'BCII1', 'BCEL', 'BCDS', 'BCDA', 'BCAR', 'INMTQ', 'BBSW', 'BBSI', 'BBRG', 'INMX', 'BBOX', 'BBNK', 'BBND', 'BBI', 'BBGI', 'BBDA', 'BBCP', 'BBBB', 'BAXS1', 'BAXS', 'INPC', 'BARZ1', 'BAMM1', 'BAGL', 'AZURQ', 'INSG', 'AXYS', 'AXTI', 'INTD', 'INTF1', 'BCGI', 'BCLI', 'AXNX', 'BCONQ', 'INDG', 'INDQA', 'BFIN', 'INEL', 'INET', 'BFC', 'BEST1', 'BERW', 'BEOSZ', 'BELM', 'BELFA', 'BEIQ', 'INFS1', 'INFT', 'BEAM1', 'BEAM', 'BDTX', 'BDSI', 'INGE', 'INHX', 'BDGE', 'BDE1', 'BDBD', 'ININ', 'BCYC', 'INIT', 'INKP', 'BCRX', 'BCOV', 'AXPH', 'AXNT', 'BFRM', 'AUXL', 'AUTO', 'AUTNY', 'AUTL', 'AUTH', 'INZY', 'AUGT', 'AUDC', 'ION', 'IONAF', 'IOTS', 'ATTU', 'ATTL', 'ATSN', 'ATSI', 'ATSG', 'ATRX1', 'ATRS1', 'ATRS', 'ATRO', 'ATRN', 'ATRI', 'ATRC', 'ATRA', 'IPAS', 'ATPG', 'IPCC', 'IPCM', 'IPCS', 'ATNY', 'INWK', 'AVCO', 'AXIHQ', 'INVX', 'INTL1', 'AXGN', 'AXDX', 'AXCA', 'AXAS', 'INTS', 'AWRE', 'AWIN', 'AWH', 'AWGI1', 'AWEB', 'AWBCQ', 'INTV1', 'INTX', 'INTZ', 'AVRO', 'AVNW', 'AVNT1', 'AVIR1', 'AVHI', 'AVGO1', 'AVGN', 'INVE', 'AVEO', 'AVDR', 'AVDO', 'AVDL1', 'AVDL', 'AVCT', 'BFRA', 'BFST', 'ATNI', 'BPTH', 'IHRT', 'BPFH', 'III', 'IIIM', 'BOWX', 'BOUT1', 'IIIN', 'BOST', 'BOSS1', 'IIIV', 'IIN', 'BORL', 'IINT', 'IISX', 'BOOM', 'BOOL', 'BONTQ', 'BONA', 'BOMN', 'IKAN', 'BOLD', 'BOJA', 'BODY', 'BOCB', 'BOBE', 'BNVIQ', 'BNTT', 'BNHN', 'BNFT', 'BPOM', 'IGOI', 'BNCN', 'BPURQ', 'BSF', 'BSET', 'BSBN', 'IFLO', 'BRY', 'BRQS', 'IFOX', 'BRP', 'BROG', 'BROD1', 'BRNC1', 'BRNC', 'BRLI1', 'IFRX', 'BRLCQ', 'BRKT', 'BRKL', 'BRKC1', 'BRIO', 'BRID', 'IGEN1', 'BREW', 'BREL', 'BREK', 'BRDR', 'IGIC', 'BRBK', 'IGNX', 'IGOC', 'BNE', 'ILND1', 'INBK', 'BKST', 'IMGN', 'BKNG1', 'BKMU', 'IMGX1', 'IMI', 'IMIC', 'BKCC', 'IMKTA', 'BJRI', 'BJICA', 'BJGP', 'BIVN', 'IMMR', 'BITI', 'BIRT', 'BIOV', 'BIOL', 'IMNY', 'IMOS', 'IMOS1', 'IMPTQ', 'IMRA', 'IMRS', 'BIDZ', 'BHBK', 'IMUX', 'IMXI', 'BGFV', 'BFYT', 'IMGC', 'BKUNQ', 'BNCL', 'BKYF', 'BNBN', 'ILOG', 'BN', 'ILPT', 'BMTI', 'BMTC', 'ILSE', 'BMRC', 'ILUM', 'ILXI', 'BMCH', 'ILXO', 'BLZE', 'BLVD', 'BLU', 'IMAL', 'IMAN', 'IMAT', 'BLPG', 'BLOG', 'BLOCA', 'BLNK', 'IMBI', 'BLMT', 'BLFS', 'IMCC', 'BLCM', 'BLBD', 'IMDZ', 'ATNX', 'ATMI', 'AGIXQ', 'AMSWA', 'AMSF', 'AMSC', 'AMRS', 'AMRN1', 'JAKK', 'AMPLQ', 'AMPH', 'AMOT', 'AMNB', 'AMMB', 'AMIS', 'AMIEQ', 'AMGIA', 'AMFIQ', 'AMFH', 'AMEV', 'AMESQ', 'AMEH', 'AMCV', 'AMCS', 'JBSS', 'AMBC', 'AMAP', 'JDAS', 'ALXO', 'ALXA', 'ALVRQ', 'ALVR', 'JEFF', 'AMSF1', 'AMSY', 'ALTM', 'AMTB', 'ANSI', 'ANPIQ', 'ANNX', 'ANMP', 'ANLY', 'ANIP', 'ANIK', 'ANIC', 'ANGO', 'ITXIQ', 'ANET1', 'ANEN', 'ANDS', 'ANDN1', 'ANDE', 'ANDB', 'IUSA', 'IVA', 'IVAC', 'ANCX', 'ANCR', 'IVANF', 'ANCCQ', 'IVII', 'ANAD', 'AMYT', 'IVISF', 'IXYS', 'AMTI', 'ALTUQ', 'ALTI', 'ANST', 'AKRO', 'AKBA', 'AKAOQ', 'AIRV', 'AIRO1', 'JPTR', 'AIQ1', 'AIPC', 'AIND', 'JRCCQ', 'JRJC', 'JRJR1', 'JRVR', 'JRVR1', 'JSBA', 'JSTN', 'AHOM', 'AHLS', 'AHII1', 'AHG', 'AHCO', 'JWEB', 'AHCI', 'AGYS', 'JWLR', 'AGTC', 'AGPH', 'AGNT', 'AGMH', 'AGLE', 'AKLMQ', 'AKTS', 'ALTH', 'AKUS', 'ALTA', 'ALT', 'ALSK', 'ALRS', 'JFU', 'JG', 'ALRC', 'JILL1', 'JIVE', 'ALOG', 'ALNT', 'ALNA', 'ALLT', 'ALLP', 'ALIM', 'JMBA', 'ALGO', 'JMDT', 'JMXI', 'ALFA1', 'JNCE', 'ALDN', 'JNP', 'ALCO', 'ALBO', 'ALBK', 'JOSB', 'JOUT', 'ALAB', 'ANSLY', 'ANSV', 'ATLO', 'ASPM', 'ASLN', 'IRGI', 'IRGTQ', 'IRIC', 'IRIDQ', 'IRIS', 'IRMC', 'IRMD', 'ASFI', 'ASEI', 'ASDS', 'ASCX', 'ASCMA', 'ASCA', 'IRWNQ', 'ASAI', 'ISCIQ', 'ARXX', 'ARXT', 'ARTX', 'ARTT', 'ARTNA', 'ARTI1', 'ARTC', 'ARST', 'ARSC1', 'ARS', 'ARRO', 'ARQT', 'ASMB', 'ASPU', 'AROW', 'ASPV', 'ATLC', 'ATISZ', 'IPEC1', 'ATHYQ', 'ATHX', 'IPHA', 'ATEX', 'ATEN1', 'IPHS', 'ATEC', 'ATCX', 'IPIX1', 'IPIXQ', 'IPMT', 'ATAX', 'ATAR1', 'IPRT', 'ATAC1', 'ASYTQ', 'IPSU', 'ASW', 'ASVI', 'ASTX1', 'ASTX', 'IQNT', 'ASTE', 'IRCP', 'ASRT', 'ASPXQ', 'ARQL', 'ARMO', 'ITRN', 'APTO', 'APTI', 'APSG1', 'APRS', 'ISYS', 'APRE', 'ITAH', 'APPS', 'APOP', 'APOG', 'APNT', 'APM', 'ITDS', 'APLT', 'APLP', 'ITEQ1', 'APIC', 'APFC', 'APEX1', 'APEX', 'ITIC', 'APEI', 'ITIG', 'APAC', 'ITOS', 'AOSL', 'ITRA', 'AOLAQ', 'ANTH', 'ANTE', 'APTM', 'APTX', 'ARLZ', 'ISTA', 'ISLE', 'ISLN', 'ARII', 'AREX', 'AREM1', 'ISPD', 'AREA', 'ARDX', 'ARDT', 'ISPH', 'ARDNA', 'ARDMQ', 'ARDI', 'ARCT1', 'ARCT', 'ARCL', 'ARCHQ', 'ARCB', 'ARBX', 'ARAY', 'ARAV', 'AQST', 'AQMS', 'ISRL', 'ISSC', 'ISSI', 'APXT', 'APWRQ', 'APWR1', 'PRGNF', 'NYFX', 'FIXX', 'PRFT', 'OSPN', 'POLGA', 'PNRG', 'PNSN', 'PNTG', 'PNWB', 'PNHNF', 'PONE', 'PONR', 'POPE', 'PORK', 'POSOQ', 'POSS', 'PNNT', 'PNFI', 'PMFG', 'PMTI', 'PMTR', 'PNCLQ', 'POVT', 'POWL', 'POZN', 'PPCO', 'PRAI', 'PRBZ', 'PRCM', 'PRCS', 'PRDS', 'PRFC1', 'PHHMQ', 'PHIIQ', 'PHTN', 'PHUN', 'PHYCQ', 'PHYN', 'PHYS1', 'PI', 'PICO', 'PIII', 'PILT', 'PHCC', 'PHAT', 'PME', 'PGC', 'PGEX', 'PGICQ', 'PGL', 'PGNS', 'PGNX', 'PGRX', 'PGS', 'PGSW', 'PLTE', 'PLPC', 'PLRE', 'PLRX', 'PLRX1', 'PLSB1', 'PLSE', 'PLUS', 'PLXT', 'PLYA', 'PMB', 'PLNR', 'PLMD', 'PIOG', 'PIRS', 'PJAM', 'PKOH', 'PKSI', 'PKT', 'PKTR', 'PLAB', 'PLAY1', 'PLFE', 'PLKI', 'PLLL1', 'PFWD', 'PCMI', 'PDGI', 'PDFS', 'PCTY1', 'OPI', 'PCTL', 'PCTI', 'PCSB', 'PCSA1', 'OPMR', 'PCOR1', 'PCOR', 'PCOM', 'OPNT1', 'PCLE', 'PDSB', 'OPRA', 'OPRT', 'OPSW', 'PCBK', 'PCBC', 'PCB', 'OPTM', 'OPTN', 'OPTN1', 'OPTR', 'PBTS', 'PBSI', 'PBSC', 'OPEN1', 'PDYN', 'PFIS', 'PERI1', 'PFGC1', 'ONEW', 'PFCO', 'PFCB', 'PFC', 'PETX', 'PETS', 'PETQ', 'PETC1', 'ONSI1', 'PESC', 'PERY', 'ONSM', 'ONTX', 'OPAY1', 'PERC', 'PEOP1', 'ONVI', 'PENX', 'PENG', 'PEIX', 'PEGS', 'ONVO', 'PEET', 'ONXS', 'PECS', 'PEBO', 'OPAY', 'PBPB', 'PBOC', 'OPTX', 'OTRK', 'OXSQ', 'OXFD', 'OSBC', 'OSCA', 'OSCI', 'OVTI', 'OSII', 'OVNT', 'OSIR', 'OVID', 'OUTD', 'OSMT', 'OTWO', 'OTIX', 'PBNC2', 'OTIC', 'OTGS', 'OTEC1', 'OSW', 'OSUR', 'OSTE', 'OYST', 'OS', 'ORXE', 'PACB', 'OPXT', 'PBKS', 'ORBC', 'ORBZ', 'ORCC1', 'PBFS', 'ORCH', 'ORCT', 'OREXQ', 'PAYS', 'ORGO', 'PATK', 'ORIC', 'PATH1', 'PASW', 'PASG', 'ORIT', 'PAND', 'PANC', 'PAIR1', 'PAGE', 'PAET', 'PACW1', 'PACT', 'PACR', 'ORTX', 'PACE1', 'ONEM1', 'PFLT', 'ODSY', 'OCPI', 'OCSI', 'OCSL', 'OCUL', 'OCZTQ', 'ODT', 'OCLR', 'OESX', 'OFIS', 'OFIX', 'OFLX', 'OGI', 'OGNC', 'OHAI', 'OCLR1', 'PFMT', 'NYNY', 'OABC', 'OAKT', 'OASM', 'OATS', 'OCHTQ', 'OBNK', 'OBSV', 'OCAT', 'OCC', 'OCFC', 'OHBIQ', 'OIIM', 'OMPI', 'OMQP', 'OMRI', 'OMTH', 'OMTR', 'ONBK', 'ONCO', 'ONCT', 'ONCY', 'OMP', 'OMNY', 'OKSB', 'OLBK', 'OMED', 'OMEF', 'OMER', 'OMKT', 'GLMC', 'LORX', 'GLMD', 'VINT', 'RUM1', 'PATH', 'WBKC1', 'NCES', 'WBNK', 'NCEM', 'RUTX', 'SIDE', 'WDHD', 'VLG', 'NGBF', 'KTIC', 'PATI', 'KZIA', 'NGCN', 'KIPS', 'KLOC', 'NGAS', 'WBKC', 'LUND', 'GSLC1', 'WFBC', 'PAUH', 'NGAC', 'GSLA', 'SMCC', 'SMCH', 'VIFL', 'OVON', 'PEGI1', 'KTIE', 'OVRL', 'GLOB1', 'PESI', 'WAYN', 'GLIAQ', 'LUNA', 'ROME', 'VLLY', 'WBCO', 'NBAC', 'ROSS', 'SINT', 'OWWI', 'WEST', 'GIGX', 'WEBK', 'GTEC1', 'KVCO', 'WEBK1', 'SIMN', 'VISL', 'ROSGQ', 'PEEK', 'PABN', 'SMIT', 'NHRX', 'PABK', 'GTIM', 'PEOP', 'RWKS', 'PACI', 'KXIN', 'GLBE', 'PACT1', 'KWIC', 'SMED', 'SMIN1', 'RWAV', 'NICK', 'VIST1', 'RVWD', 'VIVE', 'GTAX', 'PERFQ', 'NBSC', 'PERI', 'GTEC', 'NBSE', 'NBTF', 'NHMC', 'RVEN', 'GLBS', 'GTLL', 'NGTC', 'SIFI', 'SIND', 'WEDC', 'GLDC', 'NCDI', 'NCEB', 'NBN', 'PANL', 'WCAA', 'GLG', 'KTRA', 'PARL', 'SMCX', 'WCAP', 'WCBI', 'RWLK', 'NCBH', 'PERM1', 'NHLD', 'RVSB', 'SIGC', 'VIRC', 'NHHC', 'PALX', 'KKRE', 'KUTV', 'NCBM', 'GLBZ', 'PAMM', 'VKSC', 'NBAN', 'VRBA', 'LPAC', 'WAST', 'SLVR', 'SKIS', 'LODG', 'LQCI', 'GOLF1', 'LTBR', 'VSNT', 'VSNA', 'KRKR', 'GRIN', 'VSHP', 'VOXW', 'LSVI', 'GORXQ', 'VSCP', 'NETE', 'VRYAF', 'LQNT', 'SLNO', 'PCYO', 'PCYG', 'PCBI', 'NCTI', 'PCAP', 'VSTA2', 'VOLT1', 'SLTM', 'VTGN', 'PBTX', 'NEWA', 'VTEX', 'LPTX', 'VTCH', 'NCST', 'SLRY', 'NETX', 'PCAGY', 'PBYP', 'GRLLQ', 'NETR', 'GNVSQ', 'GRKA', 'PCAF', 'VSTY', 'LPWR', 'VSTN', 'SKRID', 'GOTK', 'PCCI', 'LSBX', 'GPIC', 'RSHX', 'LSBG', 'SLFC', 'PCSA', 'LSBI', 'LSBK', 'NEON', 'GPRC', 'GRCY', 'VRME', 'RSBI', 'GRCO', 'SLGG', 'VRES', 'VRAI1', 'LSCO', 'VRCC', 'KNTK', 'KOFX', 'NECB1', 'PCRV', 'VPCO', 'KOSS', 'SKYM', 'VPGC', 'RSSS', 'NEST', 'VRTB', 'SLNE', 'RSND', 'PCNTF', 'PCOL', 'SLMD', 'NECB', 'SLAM', 'NEPH', 'PCOP', 'NEPF', 'LRNS', 'LSAC', 'SLCT', 'GRDL', 'NEBS', 'GNTL', 'GNTIY', 'NFSB', 'VTNC', 'PDSN', 'GMED1', 'WALB', 'PBIO1', 'PBIP', 'NCPM', 'GMNI', 'GSCI', 'PDRT', 'VMAC', 'RPCX', 'RTNB', 'PBIX', 'PBIX1', 'PDLB', 'GRYP', 'LPDX', 'PBKB', 'VMIX', 'LTRN', 'SJ', 'GSFC', 'WALK', 'PBHC', 'GSIT', 'PAVM', 'KTHUF', 'KTEC', 'PBBI', 'RUBO', 'KTCO', 'RTWI1', 'PBCI', 'VLRX1', 'GSII', 'LTUS1', 'PECK', 'WALS', 'WALL', 'GMBL', 'PEBK', 'NFNT', 'PEAK1', 'KTCC', 'NCOC', 'GMTC', 'SJFC', 'LTONY', 'GNSS', 'RTEX', 'KNAN', 'KNAP', 'NCRI', 'SMAC', 'GNSM', 'SKFB', 'LTCW', 'VUPDA', 'VTX', 'PBOX', 'LPSB', 'LTCH', 'PBSK', 'VTVT', 'PDEX', 'PBTC', 'GRNV', 'NEWC1', 'VTNR', 'GNRS', 'VUZI', 'VNEC', 'GNLB', 'SJNB', 'GNCNF', 'GNET', 'VYFC', 'KRUS', 'RTLC', 'PBNI', 'SKBI', 'VX', 'PBNK1', 'GNPX', 'SKBO', 'GNMX', 'VNGD', 'KRSC', 'NEWZ', 'SKDS', 'VVPR', 'GNOW', 'VVID', 'OVLY', 'MDGS', 'OVID1', 'OMNI', 'FPBN', 'SEDN', 'SEEC', 'YCB', 'SEED', 'SEEQ', 'YAKC', 'SEER', 'OMEX', 'NVAL', 'SEGU', 'FPU', 'FRAF', 'XWES', 'OMTL', 'XTRM1', 'SBRG', 'XTNT1', 'OMNT', 'LEVL', 'XTEL', 'FRES', 'LESR', 'LEON1', 'SBPH', 'SEHI', 'SEMD', 'LEIX', 'LFAC', 'YECO', 'YESM', 'YFBC', 'FORK', 'FORL', 'LGBT', 'SCAN', 'YTEC', 'OHSL', 'FORU1', 'YSII', 'FOTO', 'YRKG', 'FOUR1', 'YORK', 'NVLD', 'OISI', 'NVFY', 'LIZI', 'NVET', 'FOXX', 'LFED', 'YILD', 'LFBI', 'LFAY', 'OLCB', 'OLED1', 'YGYI', 'YGTY', 'YFED', 'NTWK', 'XRDC', 'FORD', 'FSFG', 'SBCP', 'ONES', 'FSBS', 'SEV', 'FSBW', 'SEVN', 'ONFC', 'XION', 'FSEA', 'LDSD', 'FSFF', 'SEWY', 'ONHN', 'ONPR', 'FRNK', 'FSFT', 'LDMK', 'SBCO', 'XGEN', 'SFBC', 'SFBC1', 'XETA', 'SBBX', 'SFFS', 'FSNJ', 'ONTC', 'SFGH', 'FSON1', 'NTMV', 'SESN', 'LEAP1', 'NTOL', 'NTST1', 'LEGC', 'SEMIQ', 'NTSC', 'XPLR', 'SEMX', 'ONAVQ', 'XOSY', 'LMB', 'LEFX', 'SENGQ', 'FRSH1', 'NTRP', 'SENO', 'ONBI', 'FRTG1', 'FRTL', 'FRW', 'SBGA', 'LECH', 'ONCS', 'SBFG', 'FSBC2', 'FSBI', 'LEBC', 'SERA', 'ONDI', 'LIVX', 'LGHL', 'WFDS', 'LIPD', 'FLLC', 'FLMK', 'LIMS', 'LIME', 'SCLL', 'FLSCQ', 'ZNGN', 'SCPH', 'SCLD', 'LIMC', 'ZMTX', 'FLUX', 'NXTD', 'FLYR', 'FMFC', 'OCAD', 'FMARQ', 'FMBD', 'ZKIN', 'LIQB', 'LIQD', 'LIQT', 'OCGN', 'LICB', 'FMCO', 'ZIPL', 'LIBB', 'OCNB', 'FLL', 'LINUF', 'OBLN', 'OBIE', 'ZYNE', 'NYMX', 'LINE1', 'ZVXI', 'FKFS', 'ZVUE', 'NZYM', 'NXXIQ', 'SCNYB', 'OACQ', 'ZTNO', 'FLBC', 'FLBK', 'OAKF', 'ZSAN', 'LIND1', 'FLCN', 'SCOP', 'OAOT', 'SCMF', 'ZPLSQ', 'SCOT', 'ZONS', 'ZONE', 'OBAF', 'OBAS', 'OBCI', 'NWSW', 'LHMS', 'FORBQ', 'OFS', 'FNIN', 'ZANE', 'OFCP', 'FNIX', 'FNJN', 'NWCM', 'OFED', 'SCBI', 'SDII', 'FNLYQ', 'FNRG', 'FNSCQ', 'SDNA', 'ZAAP', 'OCRX', 'OGGI', 'FNWB', 'LIVE', 'NVSL', 'FOCL', 'LIVK', 'LGHS', 'OGLEQ', 'YVR', 'NVNW', 'FONR', 'FOOD2', 'FOOT', 'ZANYQ', 'LITE1', 'FNGB', 'FNFN', 'ZICA', 'FMSB', 'FMST', 'FMTIF', 'NWRZ', 'LGVN', 'ZGYH', 'FNBF', 'LGTY', 'SCEI', 'FNBG', 'FNBR', 'SCBL', 'SCYX', 'SDBT', 'FNCE', 'ODDJ', 'SDCO', 'FNCO', 'NWIR', 'NWGN1', 'NWGI', 'FNDT', 'FNET', 'ODWA', 'NWFL', 'OEG', 'XELB', 'FSPT', 'SFIN1', 'OSN', 'NPA', 'SAFE1', 'NOVS', 'SHMR', 'WLRF', 'OSIX', 'SHOEQ', 'SAESY', 'GDLN', 'GDMK1', 'WLKR', 'SACM', 'VIDA1', 'WLHN', 'GEEK', 'NOVB', 'LARS', 'OSS', 'NORC', 'WLFI', 'LARL', 'LARK', 'GECC', 'LAND1', 'GECM', 'WLDA', 'NOOF', 'NOIZ', 'OSHWQ', 'WMFG', 'LATN', 'GCLI', 'WORX', 'NRBO', 'ORRF', 'SAMA', 'GARS', 'NPTH1', 'NPTH', 'NPTE', 'ORSN', 'WOFC', 'LAZY', 'GBFE', 'SAL', 'GBI', 'WNNB', 'SAJA', 'SHIP', 'NPPI', 'NPLA', 'GBTS', 'GBTVQ', 'GBTX', 'SAFT1', 'GBUR', 'LAUN', 'GCHI', 'OSHC', 'NOEC', 'LOAC', 'FSST', 'GFLS', 'NMHC', 'WHRT', 'WHLR', 'GEXA', 'GFCO', 'LABN', 'WHLM', 'GFED', 'WHIT', 'SIBI', 'NMCI', 'NMBT', 'OUTL', 'WHCP', 'WKEY', 'GFSI', 'GGAC', 'GGBMQ', 'OVBC', 'GGNS', 'GGUY', 'WGNB', 'WGBS', 'LODE1', 'GGXY', 'LABC', 'NLCI', 'WFHCQ', 'RYOU', 'NMIL', 'WIKS', 'WILC', 'LOAN', 'LAKE', 'WIX1', 'WITM', 'WISA', 'LOAX', 'NOBLF', 'GENR', 'WIRX', 'NNDM', 'OTEL', 'NNCI', 'OTFC', 'NMTX', 'SHSO', 'NMTI', 'LAIX1', 'SHSP', 'NMRX', 'LADF', 'LACQ', 'GEOW', 'SHWC', 'OTLK', 'GEPT', 'LOCM', 'NMPC', 'WPIC', 'ORPH1', 'OROA', 'OPOF', 'LCNB', 'LCGI', 'LCCO', 'LMST', 'WWAY', 'SFT', 'FTFT', 'LCBM', 'FTHM', 'FTHR', 'OPNT', 'WVFC', 'FTRE', 'NSSY', 'SHEL', 'LCAP', 'LNBB', 'FTSW', 'OPRX', 'SGDE', 'OPTC', 'SAVB', 'OPTI', 'FUBC', 'OPTK', 'LNCB', 'WTLK', 'FULL', 'OPHM', 'NTCS', 'OPHC', 'OPES', 'FSTC1', 'XCUR', 'FSTC2', 'SFLX', 'LCSI', 'XCOMQ', 'FSTP', 'SBAY', 'LCRY', 'LMPX', 'SBAN', 'ONYX1', 'SFSL', 'LCRD', 'NTIC', 'XBKS1', 'XAIR', 'OPBK', 'XACT', 'SFSW', 'FTCG', 'LMRK1', 'WYNE', 'OPEN2', 'WWTRQ', 'LCOR', 'NTEC', 'WTHNY', 'WTER', 'FUNC', 'WSFGQ', 'ORBT', 'NRXGQ', 'NRRD', 'SGOC', 'ORCD', 'WSBI', 'SGRP', 'SANW', 'FXCB', 'ORCI', 'WSB', 'NRIM', 'ORFR', 'WRNB', 'SGVB', 'FXNC', 'WRLS1', 'ORGS', 'SGXP', 'SAMG', 'GADZQ', 'SAMB', 'GAI', 'SHBI', 'ORMP', 'SHCR', 'NRDS', 'LNCC', 'ORAL', 'SGHL', 'WSGI', 'WTCT', 'WTCO', 'FUSB', 'FUSE1', 'NSIX', 'SAUC', 'SGIC', 'FUSN1', 'WSTR1', 'FUV', 'FVCB', 'OPTO', 'FVNB', 'OPTS', 'WSTG', 'WSTF', 'WSTE1', 'OPTT', 'SGLB', 'NSHA', 'NSFC', 'NSEC', 'NSDB', 'WSIIQ', 'SGMA', 'FWFC', 'OPTT1', 'LASE', 'MYFW', 'VIBC', 'PSHG', 'ITRM', 'TMAX', 'TMAV1', 'PSID', 'STAF', 'MITY', 'TMANQ', 'MBTA', 'IHCC', 'IHSC', 'IICR', 'REIN', 'STBC', 'TLSA', 'STBI1', 'ITLN', 'MBVX', 'TLOG', 'TLNT', 'PSRC1', 'PSTA', 'STBK', 'STCL', 'STCO1', 'ITI', 'MILK', 'MIKR', 'STCR', 'PSTFY', 'TMBLQ', 'SSSS', 'SPSIQ', 'TMBS', 'PRWT', 'ITSY', 'MBRX', 'ITSW', 'SSPE', 'TMPL', 'SSPI', 'IFRS', 'PRZM1', 'PSAI', 'TMOT', 'TMOSQ', 'PSBH', 'MKTSQ', 'TMNT', 'MBRX1', 'PSBI', 'PSCDQ', 'REKR', 'IFUL', 'MKSP', 'PSCX', 'SSPX', 'PSDS', 'SSRG', 'REIS', 'IGLC', 'TMDI', 'SSRX', 'IKNX', 'PSTI', 'IKOS', 'STDY', 'TIGR1', 'MICN', 'STKS', 'REDE', 'PTIS', 'ISWI', 'MHUT', 'THTX', 'THTL', 'REDB', 'MCDY', 'IMCB', 'ISTR', 'STMDA', 'RECY', 'MGYR', 'MGXX', 'MGXI', 'THRD', 'PTNX', 'STMI', 'THNK', 'PTRS', 'REBC', 'THMO', 'THMD', 'MGRP', 'RDVT', 'PTSX', 'MCBK', 'TIMBZ', 'IMAG', 'PTAC', 'TLCX', 'PSTRQ', 'STEK', 'ILCO', 'ILFO', 'MCAC', 'STER', 'STFR', 'TLC', 'MCAR1', 'MIIX', 'MIGS', 'STGG', 'TKTL', 'STIM1', 'STGN', 'TKCI1', 'REFN', 'MIGP', 'PTEK', 'MIGI', 'TISA', 'STHKQ', 'PTET', 'MCBI1', 'IMAA', 'PTHW', 'IMAB', 'IFMK', 'IFLY1', 'TMSTA', 'MBIA', 'ICGN', 'TRCB', 'IZEA', 'TRBRQ', 'MNSB', 'TRBC', 'TRAV', 'MNRK', 'SPZE', 'PROV', 'MNPR', 'MNPQC', 'ICLK1', 'RENX1', 'PRPH', 'MBFC', 'ICMB', 'MNOC', 'ICMT', 'ICNT', 'SRAC', 'MNDO', 'PRPX', 'IVTY', 'RENO', 'ICOPQ', 'TPLQ', 'MNCL', 'IVTC', 'PROF', 'TRCH', 'TRCI', 'IBUY2', 'MAYS', 'IBEX', 'PRMO', 'TRGA', 'REST', 'JAGX', 'JADE', 'TRFX1', 'MOCO', 'JACQ', 'IBUY1', 'TRFC', 'MBBC', 'RESN', 'REPTQ', 'TREV', 'ICAD', 'ICAR', 'MNTX1', 'MBCN', 'TRED', 'ICBK', 'ICCH', 'MNTX', 'ICCXQ', 'TRDT', 'ICEL', 'SPTR', 'SRAX', 'ICOR2', 'RELL', 'SRCH1', 'ITXN', 'MLVF', 'PRTS1', 'MLRI', 'MLRC', 'IDYN', 'TNOW', 'IEA', 'IEC', 'PRTU', 'IENT', 'SSET1', 'SSFC', 'SSFN', 'IFCB', 'MLIN', 'TNCX', 'TNCC', 'PRVL1', 'SSKN', 'PRVOZ', 'IFCN', 'IFCO', 'TMXI', 'RELV', 'MBRG', 'MBRKQ', 'TMTV', 'IFLG', 'TNSU', 'TNTR', 'IDSA', 'PRTKQ', 'MMUS', 'MMTM1', 'TOWR1', 'SRCM', 'PRSNQ', 'REMX1', 'TOTL1', 'TOTA', 'TORCQ', 'SRLSQ', 'TOPS', 'SRS1', 'ITYCQ', 'SRSL', 'MBNK', 'IDEA1', 'MBND', 'MMDM', 'TOFF', 'IDGR', 'TOFC', 'IDMI', 'SRTS', 'MMCE', 'IDPIQ', 'MMAC', 'TNXP', 'TNTX', 'PTSX1', 'THCG', 'STMT', 'QTWW', 'QSND', 'TARG', 'TARA', 'SUNQ', 'MEDW', 'TANH', 'TAMB', 'SUNS', 'IROQ1', 'SUNW', 'INTXA', 'IROQ', 'MEDSE', 'MEDS', 'MCX1', 'RBZ', 'INVA2', 'TACO1', 'QTRAQ', 'INVC', 'RBOW', 'IRIX', 'RBOT', 'RBNK', 'SYSF', 'MEDP1', 'INVT', 'SURG', 'SYQT', 'TATT', 'IRW', 'TAVA', 'ISDI', 'ISEE1', 'MEMY', 'TBUSQ', 'MEMS', 'TBSI', 'RCEL', 'TBOW', 'QLNX', 'INSE', 'MCTIQ', 'INSGY', 'MEMH', 'SUMR', 'INSN', 'MCUR', 'MELR', 'ISCX', 'SUMT', 'TBHS', 'SUMX', 'TBCOA', 'ISCG', 'SUND1', 'TBAC', 'TAYD', 'INTG', 'TAVI', 'SUNN', 'SVBF', 'SVBI', 'TBWC', 'QUIN', 'MDDS', 'SXTN', 'RANGY', 'SXTC', 'IQST', 'RARB1', 'SWVA', 'SWTX1', 'IPDN', 'SVTG', 'RAVE', 'MDNU', 'IPWR', 'SWRX', 'IPET', 'MDMD', 'SWRG', 'RAWL', 'SWPA', 'SWLH', 'IPT', 'IPII', 'MDLK', 'SWCB1', 'MDKI', 'SWCB', 'SWBS', 'RAZR', 'SVVC', 'MDSYF', 'RBI', 'RBKB', 'MECN', 'QUIX', 'MDBK', 'MEDE', 'INXI', 'QUOT1', 'MDBS1', 'QUSTQ', 'IOMI', 'QVDX', 'RBLG', 'MED2', 'SYNL1', 'RACA', 'SYNL', 'RBKV', 'MECK1', 'MECH', 'SYNF', 'MDCD', 'RACE1', 'RADA', 'RADA1', 'SVLF', 'MDWY', 'MDWV', 'MDWD', 'IPAH', 'SYLN', 'INPX', 'INPHQ', 'STND', 'INFM', 'IMUL', 'PVAT', 'IMV', 'PVBC', 'MFSL', 'INAI1', 'PVCC', 'TEXM', 'INBI1', 'PVFC', 'STRM', 'PVSA', 'STRN', 'INCB', 'INCR1', 'TESI', 'MFNC', 'MFLO', 'PWCC', 'STRS', 'MFI', 'MCLN1', 'ISON', 'TENX', 'INET2', 'ISNT', 'PWFL', 'MFCB1', 'PWOD', 'TFFP', 'MGCD', 'IMTX', 'IMPC1', 'THCA', 'IMMCQ', 'IMMP', 'PUBO', 'MGNB', 'IMNPQ', 'RDHS', 'IMNT', 'RDHL', 'PULB', 'PULM', 'PULS2', 'STNV', 'STOK1', 'IMTV', 'STRC', 'IMPV1', 'TGISQ', 'PURW', 'STRD', 'MGEN', 'IMSC', 'RDGE', 'TFRC', 'TFINQ', 'IMSX', 'IMTI', 'IMTR', 'MCLS', 'MFAC1', 'INOW', 'INFN1', 'INMD1', 'SUBI', 'QDEK', 'QDIN', 'QEDC', 'INMG', 'TCON', 'TCMS', 'TCMI', 'QELP', 'TCLIF', 'TCFC', 'QFABQ', 'MERB', 'INNL', 'RCKY', 'SUFI', 'SUMA', 'ISKO', 'INOC', 'QKTN', 'INOD', 'TCAM', 'RCHY', 'ISIG', 'MENS', 'INOV1', 'QLGN', 'SUMM1', 'TCPI1', 'INMB', 'SUAI', 'PXS', 'MEXP', 'PWRH', 'ISNS', 'PWX', 'INFRA', 'TELC', 'TELA', 'TEKCQ', 'MCMJ', 'ISNR', 'TECJQ', 'RDCM', 'METF', 'PXXI', 'TCPS', 'INGNQ', 'PYMX', 'TEAM1', 'TDBK', 'PYTV', 'TDAC', 'PZZ', 'RCMT', 'MCON', 'STXX1', 'TCSI', 'INLN', 'STYL', 'TRGP1', 'IBCL1', 'LUNR', 'PHYX', 'MTXX', 'PICM', 'USAU', 'MTWK', 'MTSXY', 'HDS1', 'SNES', 'MTST1', 'PIH', 'HDVS', 'SNETE', 'PILLQ', 'HEAT1', 'HEBT', 'USAB', 'URSI', 'MTSI2', 'MTSI1', 'URMD', 'SNFCA', 'HECO1', 'SNGX', 'URGI', 'HEIIQ', 'MAGI', 'PINN', 'SNKI', 'HELO', 'PINN1', 'SNEN', 'HDIX', 'PRLX', 'HCT1', 'HBP', 'HBSC', 'KAYE', 'HCAP', 'HCAP1', 'MWAR', 'USOL', 'USMD', 'USLB1', 'RNDB', 'HCCH', 'USIO', 'PHXX', 'USHS', 'KARR', 'HCDC', 'HCEN', 'HCIA', 'KARE', 'MVII', 'MVCO', 'USEY', 'HCOW', 'USEG', 'USDL', 'HCRC', 'MVBF', 'MUZE', 'USCM', 'JYNT', 'MTRA', 'HEMT', 'HEPA', 'JVA', 'HHNT', 'MTHR', 'UNBH', 'SNSR1', 'UNB', 'UNAM', 'UMED1', 'MAMS', 'PLDI1', 'PLEN', 'JUNO1', 'PLFC', 'MANA', 'MTEC1', 'MTEC', 'HILI1', 'JUDG', 'PLIN', 'JTPY', 'SNSTA', 'HIPC1', 'UHOS', 'HIRE', 'MANC', 'UGLY', 'PLMT', 'HIRE1', 'UG', 'PLBC', 'HHCA', 'PLAY2', 'MTMS', 'JXVL', 'SNKTY', 'MTOX', 'PIONA', 'UPI', 'MTOAF', 'HETC', 'PIXY', 'HFBC', 'MAGR', 'HFBL', 'PKBK', 'HFFC', 'HFNC', 'RMPO', 'MAGS', 'JXSB', 'HGBL', 'UNTY', 'UNTKQ', 'MTMD', 'UNNL', 'SNOA', 'HGFN', 'HGGR', 'UNMG', 'HGRD', 'MTIX', 'USVI', 'HBOS', 'PHST', 'KFBI', 'VERO', 'KIDD', 'ROIQ', 'PFIN', 'ROHN', 'GXXM', 'NADX', 'VERB', 'NACT', 'VERA', 'ROCM', 'PFSB', 'GYRO1', 'MYTNQ', 'SMTB', 'MYSZ', 'MYST', 'PFSL', 'VCSI', 'VCNX', 'MYR1', 'HACH', 'VCIX', 'VCGH', 'VCFC', 'PFX1', 'KFFB', 'ROCH', 'PGDA', 'VERU', 'NAHC', 'GWRS', 'VFND', 'GTSI', 'VIAX', 'PETZ', 'PFAE', 'PFBI', 'SMPL1', 'KINS', 'NASMQ', 'SMPL2', 'GUAR', 'SMPS', 'SMPX', 'VFNX', 'PFHD', 'NAIG', 'KIDS1', 'VEXPQ', 'ROKA', 'NAME', 'GVP', 'VETS1', 'NAII', 'LVSB', 'GWBK1', 'PFIE', 'VESC', 'GWLKQ', 'VERY', 'MYOS', 'VCAT', 'HBNK1', 'VCAP', 'HBCO', 'KERN', 'KERA', 'KEQU', 'SNBP', 'MACM', 'UTOG1', 'HBEI', 'SNBT', 'PGTZ', 'KDUS', 'HBEK', 'PHAR', 'HBFW', 'HBIO', 'RNIC', 'RNHDA', 'UTCC', 'KBSF', 'PHIO', 'MWRX', 'KBLM', 'HBK', 'RNGY', 'SNDL', 'MWK', 'SNDS', 'HBNK', 'PHSB', 'UWBKQ', 'MXUS', 'VADO', 'VBAC', 'ROAC', 'PGEN1', 'MYLX', 'HAMPQ', 'VBR1', 'VBNJ', 'HAPP', 'VBLT', 'VBIV1', 'PGEO', 'LYRA', 'HAST', 'VBFC', 'HAUSQ', 'VAIL', 'KEYP', 'HAVN', 'SMXC', 'VARL', 'PGLC', 'MDII1', 'VAPHQ', 'SNAK', 'SNAP1', 'VALV', 'MABAA', 'VALN', 'RNVA', 'HIST', 'JRSH', 'SNTO', 'TSAR', 'TSTC', 'MATT', 'TSST', 'HSKL', 'TSSS', 'MATV', 'TSRI', 'MPWG', 'HSTO', 'TSRE', 'TSRC', 'SPLK1', 'TSII', 'TSICQ', 'RIBI', 'RHIEQ', 'RGSE', 'HTEI', 'HTGM', 'MPDI', 'MPB', 'PPIH', 'TSCPQ', 'PPLS', 'PPOD', 'JCS', 'MPAC1', 'PPRT', 'TSBK', 'TSTN', 'POSC', 'SPGZ', 'HRMI', 'PNTE', 'TURN', 'MRCC', 'SPCB', 'JGBO', 'SPCHB', 'PNTR', 'PNTU', 'PNVN', 'HRDG', 'MRBK', 'SPEC', 'POLA', 'JFIN', 'TSTY', 'TTIL', 'HROW', 'HRSH', 'TTII1', 'TTHI', 'JFBI', 'TTEN', 'JEVC', 'HRZN', 'MRAM', 'HSDC', 'HSDT', 'POPM', 'PPSI', 'MAXF', 'PNTA1', 'RGLOF', 'PRFN1', 'HYDGQ', 'RFIL', 'TRNS', 'SPPR', 'HYPRQ', 'TRNDA', 'HYRE', 'REXN', 'SPRI1', 'PRFN2', 'JASNQ', 'PRFS1', 'TRKN', 'TRKA', 'JASN1', 'SPRO', 'MOLGY', 'IACO', 'MOKO', 'MOIL', 'JAMS', 'MOHO', 'IASG', 'MOGO', 'REVU', 'TRIL', 'PRLS', 'IBCA', 'JAWZ', 'HYBD', 'RFMI', 'JAXR', 'TRXI', 'TRVS', 'MOXC', 'JCBSQ', 'TRVI', 'TRUE2', 'TRUE1', 'TRTL', 'RGDX', 'PRCN', 'RGCO', 'HVDC', 'MAXIQ', 'HWBK', 'TROV1', 'MOTS', 'PRCP', 'JAXB', 'HWFGQ', 'TRSI', 'SPORQ', 'PREM', 'TRPS1', 'HWLD', 'MOSX', 'MOSS', 'TROY', 'MORP', 'TUSC', 'HQI', 'UFHI', 'UACAA', 'UBSC', 'UBOH', 'RLJE', 'RKDA', 'MARA', 'MSLI', 'SOCB', 'UBMT', 'MARC', 'PMBC', 'UBIX', 'PMCOQ', 'PMD', 'HMNF', 'UBFO', 'UBET', 'HMNY', 'SOCR', 'HMP', 'MSDX', 'HMSC', 'UBCP', 'SOCT', 'HMSV', 'MARG', 'SODK', 'HMTA', 'PMIC', 'UAHC', 'RLLY', 'UCAP', 'HMGC', 'UCOR', 'PLNS', 'RMI1', 'UFAB1', 'PLPM', 'RMHT', 'HKID', 'UDYN', 'RMGN', 'HLCSQ', 'MTC', 'JRBK', 'MTBC', 'RMCF', 'SNVT', 'UCBA', 'RMBL', 'RMBI', 'MSWP', 'UCMPQ', 'PLSIQ', 'MSTG1', 'JPST1', 'PLSK', 'JPMX', 'PLUM', 'RLOGQ', 'PLXP', 'UCBC', 'JORE', 'PMII', 'MRCM', 'PMIL', 'SONW', 'TWNE', 'PNBK', 'JLNY', 'JLMI', 'SOPN', 'HOOV', 'MASC', 'HORT', 'PNDA', 'HOTH', 'PNFT', 'HOTJ1', 'TWFC', 'HPAC', 'HPAC1', 'HPBC', 'SORL', 'HPFC', 'PNNW', 'HPJ', 'MASK', 'TVINQ', 'PNO', 'MRET', 'MREO', 'SPAN', 'SPAZ', 'MATE', 'JMAR', 'HOMZQ', 'PNBF', 'MARPS', 'SOHO', 'HNIN', 'HNNA', 'SOLO', 'RIVR', 'RIVL', 'TZAC', 'PMIX', 'SOLY', 'SOMH', 'MSBF', 'SOMN', 'MARN', 'HOEN', 'HOME1', 'PMOR', 'PMRX', 'HOFV', 'RIVE', 'PMRY', 'PMSI', 'SONM', 'HOKU', 'JMBI', 'RITA', 'MARSA', 'MASB', 'MROE', 'KASP', 'AAAB', 'BKSC', 'CASM', 'CATA1', 'CATB', 'CATB1', 'CATH1', 'CATS1', 'CATZ', 'CAVB', 'CAVN', 'CAVO', 'CAYN', 'CBAN', 'CBBO', 'CBCI', 'CBDR', 'CASL', 'CASB', 'CARY', 'CALL1', 'CADMQ', 'CAER', 'CAFI', 'CALAQ', 'CALB', 'CALI', 'CAMB', 'CARV', 'CAMH', 'CAPF', 'CAPR', 'CARD', 'CARE1', 'CART', 'CBFV', 'CBIV', 'CBLI', 'CDCO1', 'CCON1', 'CCRI', 'CCRO', 'CCSI', 'CCTX', 'CDCM', 'CDCSY', 'CCLN', 'CDCY', 'CDDD', 'CDEN', 'CDII', 'CDIR', 'CDMS', 'CCNC', 'CCHM', 'CBLT', 'CBNK1', 'CBMB', 'CBMD', 'CBMX', 'CBNJ', 'CBNJ1', 'CBNK', 'CBRYA', 'CCDC', 'CBSA1', 'CBUK', 'CCB', 'CCBD', 'CCBP', 'CCBT', 'CADE2', 'CADA', 'CACGU', 'BOXL', 'BONS', 'BONZ1', 'BOOT1', 'BORD', 'BOSA', 'BOTJ', 'BOYL', 'BOLD1', 'BPAO', 'BPLS', 'BPLX', 'BPTM', 'BPTR', 'BRANF', 'BOLT', 'BOFLQ', 'BRDG', 'BMED1', 'BLCT', 'BLMT1', 'BLPH', 'BLRX', 'BLZN', 'BMAN', 'BMRA', 'BOCH', 'BMRQ', 'BNBC', 'BNCM1', 'BNGO', 'BNSSA', 'BOBS', 'BRCP', 'BREW1', 'CABL', 'BXRX', 'BUTLQ', 'BVSI', 'BWAY', 'BWFG', 'BWSI', 'BWTR', 'BXX', 'BUR', 'BYBI', 'BYBK', 'BYFC', 'BYLK', 'BZBC', 'CABG', 'BUTI', 'BULL', 'BRLI', 'BSVN', 'BRPA', 'BSBK', 'BSGM', 'BSMD', 'BSMT1', 'BSPM', 'BTBT', 'BTUI', 'BTGI', 'BTGL', 'BTIC', 'BTRN', 'BTRY', 'BTSR', 'CDOT', 'CDSW', 'CDTI', 'CMIV', 'CMDN', 'CMDZQ', 'CMED', 'CMETS', 'CMGT', 'CMIN', 'CMPI', 'CMBC', 'CMPP', 'CMPS1', 'CMPX', 'CMRN', 'CMRO', 'CMSS1', 'CMCI1', 'CMALQ', 'CNBB', 'CLIR', 'CLBR', 'CLBS', 'CLCP', 'CLDB', 'CLEV1', 'CLGN', 'CLPS', 'CMAG', 'CLPT1', 'CLRB', 'CLRO', 'CLSK', 'CLSN', 'CLYS', 'CMSV', 'CNBD', 'CLAS', 'COBK', 'CNTY', 'CNVO', 'CNXR', 'CNYD', 'CNYF', 'COA1', 'COBR', 'CNST1', 'COCN', 'COCP', 'CODA', 'COFS', 'COHB', 'COHO1', 'CNSW', 'CNSP1', 'CNBF', 'CNGL1', 'CNBI2', 'CNBT', 'CNDO', 'CNDR', 'CNET', 'CNFR', 'CNIT1', 'CNSP', 'CNKT', 'CNNG', 'CNOB1', 'CNRS', 'CNSI', 'CNSO', 'CLBH', 'CLAR1', 'CDTX', 'CFK', 'CFCB', 'CFCI', 'CFFC', 'CFFI', 'CFGI1', 'CFIC1', 'CFMT', 'CFBI1', 'CFRX', 'CFSB', 'CFSB1', 'CFSL', 'CFTP', 'CGCA', 'CFBK', 'CFBI', 'CGEN1', 'CEMI', 'CDWI', 'CEBC', 'CEBK', 'CECX1', 'CELC', 'CELM', 'CENL', 'CFBC1', 'CERC', 'CERG', 'CERP', 'CETX', 'CEXE', 'CFAC', 'CGCP1', 'CGGI', 'CLAC1', 'CIDM', 'CHNL', 'CHOPF', 'CHRI1', 'CHRK', 'CHRT1', 'CIBN1', 'CIMT', 'CHMG', 'CITZ', 'CIVC', 'CIZN', 'CJBK', 'CJJD', 'CKPT', 'CHMP', 'CHMD1', 'CGII1', 'CHAR', 'CGIX', 'CGNT', 'CGRM', 'CGRO', 'CGUE', 'CHANF', 'CHAS', 'CHLN1', 'CHERB', 'CHEV', 'CHEZ', 'CHFI', 'CHFS', 'CHLN', 'BLCI', 'BKRSQ', 'COMB1', 'BKFR', 'ALGI', 'ALGR', 'ALJJ', 'ALLB', 'ALLI', 'ALNC', 'ALNK', 'ALOT', 'ALOY', 'ALPH', 'ALPN', 'ALRN', 'ALRN2', 'ALSM', 'ALTA1', 'ALFC1', 'ALDX', 'ALDA', 'AINN', 'AICX', 'AIH', 'AIHS', 'AIKI', 'AILP', 'AIMM', 'AIRG', 'ALCS', 'AIRS', 'AIRT', 'AKER', 'ALAC', 'ALCD', 'ALCI', 'ALUS1', 'ALYA', 'ALYNQ', 'ANDA', 'AMV', 'AMXC', 'AMXI', 'ANBC', 'ANCB', 'ANCX1', 'ANDA2', 'AMSI', 'ANET2', 'ANFI1', 'ANGN1', 'ANIX', 'ANLG', 'ANNB', 'AMTX', 'AMRK', 'ALYS', 'AMCN1', 'AMAC', 'AMBC1', 'AMBI', 'AMBTQ', 'AMCF', 'AMCI', 'AMCP', 'AMRB', 'AMCT', 'AMHC', 'AMHI', 'AMIC', 'AMPH1', 'AMPXQ', 'AHWY', 'AHTC', 'AHPI', 'ACCD1', 'ABTE', 'ABTI', 'ABVA', 'ABWG', 'ACAM1', 'ACBA', 'ACEC', 'ABPI', 'ACEV', 'ACFC', 'ACHI', 'ACHV', 'ACLO', 'ACNAQ', 'ABTC']

top_other_name = "<other>"
unique_values = set(TOP_2020 + [top_other_name])

def set_ticker_tag(ticker):
    return ticker if ticker in TOP_2020 else top_other_name

f22_ticker_tag = "f22_ticker_tag"
df_tweet_counted[f22_ticker_tag] = df_tweet_counted["f22_ticker"].apply(set_ticker_tag)

df_twitter_tick_hot = ticker_service.make_one_hotted_for_one_column(df=df_tweet_counted, unique_values=unique_values, col=f22_ticker_tag)

In [ ]:
import gc

df_4_model = None
df = None
df_days_until = None
df_pay = None
df_since_sma = None
df_after_hours = None
df_green_hornet = None
df_no_z = None
df_thin_rabbit = None
df_merged = None
df_sqi = None
df_stock_quart_info = None
df_nas_tickers_info = None
df_drop_future = None
df_eq_fun_quarters = None
df_equity_fundies = None
df_rec_quart_drop = None
df_twitter_raw = None

gc.collect()

df = df_twitter_tick_hot#.copy() #.sample(frac=.4)

device = initialize_gpu()
desired_dtype = 'float64'

num_iterations = 1000

zero_in = True
if zero_in:
        target_roi_frac = 0.07
else:
    target_roi_frac = 0.001

sac_by_bucket = []
sac_list = []
for i in range(num_iterations):
    sd = split_off_data(df=df, narrow_cols=narrow_cols)
    
    if sd and sd.has_enough_data:
        X_train = sd.X_train
        y_train = sd.y_train
        X_test = sd.X_test
        y_test = sd.y_test
        df_test_raw = sd.df_test_raw
        df_test_std = sd.df_test_std
        df_val_raw = sd.df_val_raw
        df_val_std = sd.df_val_std
        train_cols = sd.train_cols
        has_enough_data = sd.has_enough_data

        if df_val_raw.shape[0] < 1000 or not has_enough_data:
            continue

        X_train_con = X_train.astype(desired_dtype)

        y_train = np.where(y_train==-1, 0, y_train)
        y_train_con = y_train.astype(desired_dtype)

        train_dataset = trainData(torch.FloatTensor(X_train_con), 
                               torch.FloatTensor(y_train_con))

        X_test_con = X_test.astype(desired_dtype)

        y_test = np.where(y_test==-1, 0, y_test)
        y_test_con = y_test.astype(desired_dtype)

        test_dataset = testData(torch.FloatTensor(X_test_con), 
                               torch.FloatTensor(y_test_con))

        train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        test_loader = DataLoader(dataset=test_dataset, batch_size=1)

        model, criterion, optimizer = get_initialized_model(num_input_features=X_train.shape[1], device=device)

        best_model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, criterion=criterion, optimizer=optimizer)

        sac_mean_list = calc_nn_roi(df_val_raw=df_test_raw, 
            df_val_std=df_test_std, 
            model=best_model, 
            train_cols=train_cols, 
            target_roi_frac=target_roi_frac,
            zero_in=zero_in
           )

        sac_list += sac_mean_list

        sac_by_bucket.append(sac_mean_list)

        print(f"\nmultiday: s@close_mean: {mean(sac_list)}\n")
    else:
        print("No data from split_off_data.")
                
    df, has_remaining_days = twitter_service.remove_last_days(df=df, num_days=1)
    if not has_remaining_days:
        print("No more remaining days to test.")
        break

Using GPU
Split | min: 2020-08-10 | max: 2020-10-23
Split dates for small dataset: ['2020-10-22', '2020-10-23']
Split | min: 2020-08-10 | max: 2020-10-21
Split dates for small dataset: ['2020-10-07', '2020-10-08', '2020-10-09', '2020-10-10', '2020-10-11', '2020-10-12', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-17', '2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21']
Original: 601796; train_set: 418690; test_set: 160164
Epoch 001: | Loss: 12.50419 | Acc: 65.689
Epoch time elapsed: 50.8011s

Test loss: 1.1645 | Acc: 57.96% | Count_loops: 156276)

Epoch 002: | Loss: 0.42600 | Acc: 75.761
Epoch time elapsed: 50.5159s

Test loss: 2.2107 | Acc: 52.98% | Count_loops: 156276)

Epoch 003: | Loss: 0.37590 | Acc: 78.765
Epoch time elapsed: 50.2633s

Test loss: 2.5979 | Acc: 53.81% | Count_loops: 156276)

Epoch 004: | Loss: 0.35140 | Acc: 80.429
Epoch time elapsed: 50.586s

Test loss: 2.6747 | Acc: 53.40% | Count_loops: 156276)

Epoch 005: | Loss: 0.33708 | Acc: 81.596
E

In [ ]:
file_path_str = str(Path(constants.OVERFLOW_DATA_PATH, "twitter", "inference_model_drop", "train_ready_tweets.csv"))

df_tweet_counted.to_csv(file_path_str, index=False)